<p>
<img src="http://www.cerm.unifi.it/chianti/images/logo%20unifi_positivo.jpg" 
        alt="UniFI logo" style="float: left; width: 20%; height: 20%;">
<div align="right">
Massimo Nocentini<br>
<small>
<br>January 30, 2017: refactoring
<br>September 12, 2016: search hints, table data representation
<br>September 7, 2016: module encapsulation, refactoring and filtering
<br>September 6, 2016: pretty printer
<br>September 5, 2016: big bang
</small>
</div>
</p>
<br>
<div align="center">
<b>Abstract</b><br>
A <i>pretty printer</i> for OEIS search result, make them inline in the notebook.
</div>

# A _pretty printer_ for OEIS search results

Our idea to integrate OEIS search results in a working notebook lies on a existing attempt to talk with http://oeis.org, under review in [this Stack Overflow thread][stack:overflow]. Evaluate the following cell to work with most recent definitions according to version history:

[stack:overflow]:http://codereview.stackexchange.com/questions/120102/an-oeis-lookup-tool-in-python

In [1]:
from oeis import search, pretty_print

Instead, for *debug and develop* sessions evaluate the following one:

In [53]:
%run ../src/oeis.py

## Search hints

OEIS provide a [page full of search hints][hints]. In order to gain performance and search capabilities already provided by the online encyclopedia, we show how to use those hints with our implementation.

The following content is kept from the *Advanced search syntax* section of the [reference page][hints]:
   - **Examples of searches**
   
        ```
        1,4,9,16,25,36,64
        5 8 13 233 39088169
        "fermat's little theorem"
        author:Guy keyword:nice
        keyword:nice keyword:more -keyword:base
        keyword:new -keyword:base
        id:A64413
        A64413
        id:A028284|id:A066948
        author:njas keyword:less|keyword:dumb
        ```
        
        - The search text is a list of space-separated search terms, as with popular web search engines.
        - The search terms can be single numbers, sequences of numbers, words or strings of words.
        - Sequences of numbers should be separated with commas.
        - Strings of words should be separated by spaces (NOT commas).
        - Strings of words may be enclosed in double quotes.
        - You can separate search terms with | (no spaces around the |) and it means sequences that match either term.
   
   - **Commas versus spaces** <br>
        Commas and spaces have different meanings. These two searches produce different results:
        
        ```
        1,4,9,16,25,36,49,64,81,100
        1 4 9 16 25 36 49 64 81 100
        ```

        The first matches sequences in which the numbers appear consecutively, in the order given.<br>
        The second matches sequences containing all the numbers, in any order. <br>
        Both searches will match the squares (A000290), but the second will also match other sequences,
        such as the composite numbers (A002808), which contain all those numbers but not necessarily consecutively or in the given order.<br>
        The same is true for word searches. Suppose you want to find sequences that cite Stanley's book Enumerative Combinatorics.<br>
        The search string Stanley, Combinatorics (with the comma) gets no hits, but searching for Stanley Combinatorics (without the comma) gets several hundred hits.
        
   - **Prefixes**<br>
    Simple search terms will be matched against every field of the entry, 
    though fields such as the name and the sequence data are given more weight than others.
    Prefixes restrict the search to particular lines, as in:

        ```
        keyword:nice author:LeBrun
        ```

    The prefixes mostly correspond to the lines in the display, as follows:

        ```
        id:                     ref:                    program:
        seq:                    link:                   xref:
        signed:                 formula:                keyword:
        name:                   example:                author:
        offset:                 maple:                  extension:
        comment:                mathematica:
        ```
        
        - `id:A001006` displays sequence A001006. A001006 displays sequence A001006 
            followed by all other sequences that mention it.
        - `seq:` matches the sequence data, possibly by ignoring signs.
        - `signed:` matches sequence data and requires that any signs match as well. 
            For example: `signed:1,-1,-1,0,-1,1,-1,0,0,1,-1,0,-1,1,1,0`
            will find the Moebius function mu(n), A008683.
        - Put a minus sign in front of a prefix to exclude those matches.
            Put a tilde (~) in front of a number to exclude it (only applies if the 
            numbers are separated by spaces). For example, a search for 2 4 ~8 16 returns 
            sequences such as 0, 1, 2, 4, 16, 65536 that do not contain 8.
        - You can say `prefix:` by itself to match sequences that have a line of that type. 
            For example `ref:` will match all sequences with reference lines.
        - Another example: you can say, e.g. `-seq:5` in your search to exclude all 
            sequences containing the number 5. So if you know your sequence consists only 
            of 1, 2, 3, and 4 (and does include all of them), you could search 
            `seq:1 seq:2 seq:3 seq:4 -seq:5 -seq:6 -seq:0`.

    Two other prefixes are concerned with subsequences:
        
        ```
        subseq:                 signedsubseq:
        ```

    You can say, for example, `subseq:377,8` to get those numbers in sequence data in that order.
   
   - **Wild cards**<br>
        - The wildcard \_ will match any number in a sequence, as in 1,2,_,4,5
        - The wildcard \__ will match any list of numbers in a sequence, as in 1,2,__,7,8
        - Search terms are highlighted in the results
        
   - **Punctuation**<br>
     is not indexed and is treated as identical to whitespace during matching;
     for example, searching for fermat's is the same as searching for "fermat s".
     
   - **Further examples**<br>
     To find sequences containing both terms 1759 and 1957: search for `seq:1759 seq:1957`.



[hints]:https://oeis.org/hints.html

## Use cases

### Search by sequence `id`entifier

In [2]:
searchable = search(A_id='A000045')

●

In [3]:
searchable(data_only=True, comment=lambda i,c: "binomial" in c)

_Results for query: <a href='https://oeis.org/search?fmt=json&start=0&q=id%3AA000045'>https://oeis.org/search?fmt=json&start=0&q=id%3AA000045</a>_<br><hr><div align='center'><b><a href='http://oeis.org/A000045'>A000045</a></b>: Fibonacci numbers: F(n) = F(n-1) + F(n-2) with F(0) = 0 and F(1) = 1.<br></div>

by _N. J. A. Sloane_, 1964

_Keywords_: `nonn,core,nice,easy,hear`

_Data_:

$$
\begin{array}{c|ccccccccccccccc }
    n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
    \hline
    A000045(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 13 & 21 & 34 & 55 & 89 & 144 & 233 & 377
\end{array}
$$
        

In [4]:
searchable(comment=lambda i,c: "binomial" in c)

_Results for query: <a href='https://oeis.org/search?fmt=json&start=0&q=id%3AA000045'>https://oeis.org/search?fmt=json&start=0&q=id%3AA000045</a>_<br><hr><div align='center'><b><a href='http://oeis.org/A000045'>A000045</a></b>: Fibonacci numbers: F(n) = F(n-1) + F(n-2) with F(0) = 0 and F(1) = 1.<br></div>

by _N. J. A. Sloane_, 1964

_Keywords_: `nonn,core,nice,easy,hear`

_Data_:

$$
\begin{array}{c|ccccccccccccccc }
    n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
    \hline
    A000045(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 13 & 21 & 34 & 55 & 89 & 144 & 233 & 377
\end{array}
$$
        

_Comments_:
   - Fib(n+2) = Sum_{k=0..n} binomial(floor((n+k)/2),k), row sums of <a href="http://oeis.org/A046854">A046854</a>. - _Paul Barry_, Mar 11 2003
   - The sequence F(n) is the binomial transformation of the alternating sequence (-1)^(n-1)*F(n), whereas the sequence F(n+1) is the binomial transformation of the alternating sequence (-1)^n*F(n-1). Both of these facts follow easily from the equalities a(n;1)=F(n+1) and b(n;1)=F(n) where a(n;d) and b(n;d) are so-called "delta-Fibonacci" numbers as defined in comments to <a href="http://oeis.org/A014445">A014445</a> (see also the papers of Witula et al.). - _Roman Witula_, Jul 24 2012
   - From _Tom Copeland_, Nov 02 2014: 
       - Let P(x) = x/(1+x) with comp. inverse Pinv(x) = x/(1-x) = -P[-x], and C(x)= [1-sqrt(1-4x)]/2, an o.g.f. for the shifted Catalan numbers <a href="http://oeis.org/A000108">A000108</a>, with inverse Cinv(x) = x * (1-x).
       - Fin(x) = P[C(x)] = C(x)/[1 + C(x)] is an o.g.f. for the Fine numbers, <a href="http://oeis.org/A000957">A000957</a> with inverse Fin^(-1)(x) = Cinv[Pinv(x)] = Cinv[-P(-x)].
       - Mot(x) = C[P(x)] = C[-Pinv(-x)] gives an o.g.f. for shifted <a href="http://oeis.org/A005043">A005043</a>, the Motzkin or Riordan numbers with comp. inverse Mot^(-1)(x) = Pinv[Cinv(x)] = (x - x^2) / (1 - x + x^2) (cf. <a href="http://oeis.org/A057078">A057078</a>).
       - BTC(x) = C[Pinv(x)] gives <a href="http://oeis.org/A007317">A007317</a>, a binomial transform of the Catalan numbers, with BTC^(-1)(x) = P[Cinv(x)].
       - Fib(x) = -Fin[Cinv(Cinv(-x))] = -P[Cinv(-x)] = x + 2 x^2 + 3 x^3 + 5 x^4 + ... = (x+x^2)/[1-x-x^2] is an o.g.f. for the shifted Fibonacci sequence <a href="http://oeis.org/A000045">A000045</a>, so the comp. inverse is Fib^(-1)(x) = -C[Pinv(-x)] = -BTC(-x) and Fib(x) = -BTC^(-1)(-x).
       - Generalizing to P(x,t) = x /(1 + t*x) and Pinv(x,t) = x /(1 - t*x) = -P(-x,t) gives other relations to lattice paths, such as the o.g.f. for <a href="http://oeis.org/A091867">A091867</a>, C[P[x,1-t]], and that for <a href="http://oeis.org/A104597">A104597</a>, Pinv[Cinv(x),t+1].

_Formulae_:
   - G.f.: x / (1 - x - x^2).
   - G.f.: Sum_{n>=0} x^n * Product_{k=1..n} (k + x)/(1 + k*x). - _Paul D. Hanna_, Oct 26 2013
   - F(n) = ((1+sqrt(5))^n-(1-sqrt(5))^n)/(2^n*sqrt(5)).
   - Alternatively, F(n) = ((1/2+sqrt(5)/2)^n-(1/2-sqrt(5)/2)^n)/sqrt(5).
   - F(n) = F(n-1) + F(n-2) = -(-1)^n F(-n).
   - F(n) = round(phi^n/sqrt(5)).
   - F(n+1) = Sum_{j=0..[n/2]} binomial(n-j, j).
   - A strong divisibility sequence, that is, gcd(a(n), a(m)) = a(gcd(n, m)) for all positive integers n and m. - _Michael Somos_, Jan 03 2017
   - E.g.f.: (2/sqrt(5))*exp(x/2)*sinh(sqrt(5)*x/2). - _Len Smiley_, Nov 30 2001
   - [0 1; 1 1]^n [0 1] = [F(n); F(n+1)]
   - x | F(n) ==> x | F(kn).
   - A sufficient condition for F(m) to be divisible by a prime p is (p - 1) divides m, if p == 1 or 4 (mod 5); (p + 1) divides m, if p == 2 or 3 (mod 5); or 5 divides m, if p = 5. (This is essentially Theorem 180 in Hardy and Wright.) - Fred W. Helenius (fredh(AT)ix.netcom.com), Jun 29 2001
   - a(n)=F(n) has the property: F(n)*F(m) + F(n+1)*F(m+1) = F(n+m+1). - _Miklos Kristof_, Nov 13 2003
   - _Kurmang. Aziz. Rashid_, Feb 21 2004, makes 4 conjectures and gives 3 theorems:
   - Conjecture 1: for n>=2 sqrt{F(2n+1)+F(2n+2)+F(2n+3)+F(2n+4)+2*(-1)^n}={F(2n+1)+2*(-1)^n}/F(n-1). [For a proof see Comments section.]
   - Conjecture 2: for n>=0, {F(n+2)* F(n+3)}-{F(n+1)* F(n+4)}+ (-1)^n = 0.
   - Conjecture 3: for n>=0, F(2n+1)^3 - F(2n+1)*[(2*A^2)-1] - [A + A^3]=0, where A = {F(2n+1)+sqrt{5*F(2n+1)^2 +4}}/2.
   - Conjecture 4: for x>=5, if x is a Fibonacci number >= 5 then g*x*[{x+sqrt{5*(x^2) +- 4}}/2]*[2x+{{x+sqrt{5*(x^2) +- 4}}/2}]*[2x+{{3x+3*sqrt {5*(x^2) +- 4}}/2}]^2+[2x+{{x+sqrt{5*(x^2) +- 4}}/2}] +- x*[2x+{{3x+3*sqrt{5*(x^2) +- 4}}/2}]^2 -x*[2x+{{x+sqrt{5*(x^2) +- 4}}/2}]*[x+{{x+sqrt{5*(x^2) +- 4}}/2}]* [2x+{{3x+3*sqrt{5*(x^2) +- 4}}/2}]^2= 0, where g = {1 + sqrt(5)/2}.
   - Theorem 1: for n>=0, {F(n+3)^ 2 - F(n+1)^ 2}/F(n+2)={F(n+3)+ F(n+1)}.
   - Theorem 2: for n>=0, F(n+10) = 11*F(n+5) + F(n).
   - Theorem 3: for n>=6, F(n) = 4*F(n-3) + F(n-6).
   - Conjecture 2 of Rashid is actually a special case of the general law F(n)*F(m) + F(n+1)*F(m+1) = F(n+m+1) (take n <- n+1 and m <- -(n+4) in this law). - Harmel Nestra (harmel.nestra(AT)ut.ee), Apr 22 2005
   - Conjecture 2 of Rashid Kurmang simplified: F(n)*F(n+3) = F(n+1)*F(n+2)-(-1)^n. Follows from d'Ocagne's identity: m=n+2. - _Alex Ratushnyak_, May 06 2012
   - Conjecture: for all c such that 2-Phi <= c < 2*(2-Phi) we have F(n) = floor(Phi*a(n-1)+c) for n > 2. - _Gerald McGarvey_, Jul 21 2004
   - |2*Fib(n) - 9*Fib(n+1)| = 4*<a href="http://oeis.org/A000032">A000032</a>(n) + <a href="http://oeis.org/A000032">A000032</a>(n+1). - _Creighton Dement_, Aug 13 2004
   - For x > Phi, Sum_{n=0..inf} F(n)/x^n = x/(x^2 - x - 1) - _Gerald McGarvey_, Oct 27 2004
   - F(n+1) = exponent of the n-th term in the series f(x, 1) determined by the equation f(x, y) = xy + f(xy, x). - _Jonathan Sondow_, Dec 19 2004
   - a(n-1) = Sum_{k=0..n} (-1)^k*binomial(n-ceil(k/2), floor(k/2)). - _Benoit Cloitre_, May 05 2005
   - F(n+1) = Sum_{k=0..n} binomial((n+k)/2, (n-k)/2)(1+(-1)^(n-k))/2. - _Paul Barry_, Aug 28 2005
   - Fibonacci(n) = Product(1 + 4[cos(j*Pi/n)]^2, j=1..ceil(n/2)-1). [Bicknell and Hoggatt, pp. 47-48.] - _Emeric Deutsch_, Oct 15 2006
   - F(n) = 2^-(n-1)*Sum_{k=0..floor((n-1)/2)} binomial(n,2*k+1)*5^k. - _Hieronymus Fischer_, Feb 07 2006
   - a(n) = (b(n+1)+b(n-1))/n where {b(n)} is the sequence <a href="http://oeis.org/A001629">A001629</a>. - _Sergio Falcon_, Nov 22 2006
   - F(n*m) = Sum_{k = 0..m} binomial(m,k)*F(n-1)^k*F(n)^(m-k)*F(m-k). The generating function of F(n*m) (n fixed, m = 0,1,2...) is G(x) = F(n)*x / ((1-F (n-1)*x)^2-F(n)*x*(1-F(n-1)*x)-( F(n)*x)^2). E.g., F(15) = 610 = F(5*3) = binomial(3,0)* F(4)^0*F(5)^3*F(3) + binomial(3,1)* F(4)^1*F(5)^2*F(2) + binomial(3,2)* F(4)^2*F(5)^1*F(1) + binomial(3,3)* F(4)^3*F(5)^0*F(0) = 1*1*125*2 + 3*3*25*1 + 3*9*5*1 + 1*27*1*0 = 250 + 225 + 135 + 0 = 610. - _Miklos Kristof_, Feb 12 2007
   - From _Miklos Kristof_, Mar 19 2007: 
       -   Let L(n) = <a href="http://oeis.org/A000032">A000032</a>(n) = Lucas numbers. Then:
       -   For a>=b and odd b, F(a+b)+F(a-b)=L(a)*F(b).
       -   For a>=b and even b, F(a+b)+F(a-b)=F(a)*L(b).
       -   For a>=b and odd b, F(a+b)-F(a-b)=F(a)*L(b).
       -   For a>=b and even b, F(a+b)-F(a-b)=L(a)*F(b).
       -   F(n+m)+(-1)^m*F(n-m)=F(n)*L(m);
       -   F(n+m)-(-1)^m*F(n-m)=L(n)*F(m);
       -   F(n+m+k)+(-1)^k*F(n+m-k)+(-1)^m*(F(n-m+k)+(-1)^k*F(n-m-k)) =F(n)*L(m)*L(k);
       -   F(n+m+k)-(-1)^k*F(n+m-k)+(-1)^m*(F(n-m+k)-(-1)^k*F(n-m-k)) =L(n)*L(m)*F(k);
       -   F(n+m+k)+(-1)^k*F(n+m-k)-(-1)^m*(F(n-m+k)+(-1)^k*F(n-m-k)) =L(n)*F(m)*L(k);
       -   F(n+m+k)-(-1)^k*F(n+m-k)-(-1)^m*(F(n-m+k)-(-1)^k*F(n-m-k)) =5*F(n)*F(m)*F(k). 
   - A corollary to Kristof 2007 is 2*F(a+b)=F(a)*L(b)+L(a)*F(b). - _Graeme McRae_, Apr 24 2014
   - For n>m, the sum of the 2m consecutive Fibonacci numbers F(n-m-1) thru F(n+m-2) is F(n)*L(m) if m is odd, and L(n)*F(m) if m is even (see the McRae link). - _Graeme McRae_, Apr 24 2014.
   - Fib(n) = b(n)+(p-1)*Sum_{1<k<n} floor(b(k)/p)*Fib(n-k+1) where b(k) is the digital sum analog of the Fibonacci recurrence, defined by b(k)=ds_p(b(k-1))+ds_p(b(k-2)), b(0)=0, b(1)=1, ds_p=digital sum base p. Example for base p=10: Fib(n)=<a href="http://oeis.org/A010077">A010077</a>(n)+9*Sum_{1<k<n} <a href="http://oeis.org/A059995">A059995</a>(<a href="http://oeis.org/A010077">A010077</a>(k))*Fib(n-k+1). - _Hieronymus Fischer_, Jul 01 2007
   - Fib(n) = b(n)+p*Sum_{1<k<n} floor(b(k)/p)*Fib(n-k+1) where b(k) is the digital product analog of the Fibonacci recurrence, defined by b(k)=dp_p(b(k-1))+dp_p(b(k-2)), b(0)=0, b(1)=1, dp_p=digital product base p. Example for base p=10: Fib(n)=<a href="http://oeis.org/A074867">A074867</a>(n)+10*Sum_{1<k<n} <a href="http://oeis.org/A059995">A059995</a>(<a href="http://oeis.org/A074867">A074867</a>(k))*Fib(n-k+1). - _Hieronymus Fischer_, Jul 01 2007
   - a(n) = denominator of continued fraction [1,1,1,...] (with n ones); e.g., 2/3 = continued fraction [1,1,1]; where barover[1] = [1,1,1...] = 0.6180339.... - _Gary W. Adamson_, Nov 29 2007
   - F(n + 3) = 2F(n + 2) - F(n), F(n + 4) = 3F(n + 2) - F(n), F(n + 8) = 7F(n + 4) - F(n), F(n + 12) = 18F(n + 6) - F(n). - _Paul Curtz_, Feb 01 2008
   - 1 = 1/(1*2) + 1/(1*3) + 1/(2*5) + 1/(3*8) + 1/(5*13) + ... = 1/2 + 1/3 + 1/10 + 1/24 + 1/65 + 1/168 + ...; where <a href="http://oeis.org/A059929">A059929</a> = (0, 2, 3, 10, 24, 65, 168,...). - _Gary W. Adamson_, Mar 16 2008
   - a(2^n) = prod{i=0}^{n-2}B(i) where B(i) is <a href="http://oeis.org/A001566">A001566</a>. Example 3*7*47 = Fib(16). - _Kenneth J Ramsey_, Apr 23 2008
   - F(n) = (1/(n-1)!) * (n^(n-1) - (C(n-2,0) + 4*C(n-2,1) + 3*C(n-2,2))*n^(n-2) + (10*C(n-3,0) + 49*C(n-3,1) + 95*C(n-3,2) + 83*C(n-3,3) + 27*C(n-3,4))*n^(n-3) - (90*C(n-4,0) + 740*C(n-4,1) + 2415*C(n-4,2) + 4110*C(n-4,3) + 3890*C(n-4,4) + 1950*C(n-4,5) + 405*C(n-4,6))*n^(n-4) + ... ). - _André F. Labossière_, Nov 24 2004
   - a(n+1) = Sum_{k, 0<=k<=n} <a href="http://oeis.org/A109466">A109466</a>(n,k)*(-1)^(n-k). -_Philippe Deléham_, Oct 26 2008
   - a(n) = Sum_{l_1=0..n+1} Sum_{l_2=0..n}...Sum_{l_i=0..n-i}... Sum_{l_n=0..1} delta(l_1,l_2,...,l_i,...,l_n), where delta(l_1,l_2,...,l_i,...,l_n) = 0 if any l_i + l_(i+1) >= 2 for i=1..n-1 and delta(l_1,l_2,...,l_i,...,l_n) = 1 otherwise. - _Thomas Wieder_, Feb 25 2009
   - a(n+1) = 2^n sqrt(Product_{k=1..n} cos(k Pi/(n+1))^2+1/4)) (Kasteleyn's formula specialized). - Sarah-Marie Belcastro (smbelcas(AT)toroidalsnark.net), Jul 04 2009
   - a(n+1) = Sum_{k=floor[n/2] mod 5} C(n,k) - Sum_{k=floor[(n+5)/2] mod 5} C(n,k) = <a href="http://oeis.org/A173125">A173125</a>(n) - <a href="http://oeis.org/A173126">A173126</a>(n) = |<a href="http://oeis.org/A054877">A054877</a>(n)-<a href="http://oeis.org/A052964">A052964</a>(n-1)|. - _Henry Bottomley_, Feb 10 2010
   - If p[i]=modp(i,2) and if A is Hessenberg matrix of order n defined by: A[i,j]=p[j-i+1], (i<=j), A[i,j]=-1, (i=j+1), and A[i,j]=0 otherwise. Then, for n>=1, a(n)=det A. - _Milan Janjic_, May 02 2010
   - Limit(F(k+n)/F(k), k = infinity) = (L(n) + F(n)*sqrt(5))/2 with the Lucas numbers L(n)= <a href="http://oeis.org/A000032">A000032</a>(n). - _Johannes W. Meijer_, May 27 2010
   - For n>=1, F(n)=round(log_2(2^{phi*F(n-1)} + 2^{phi*F(n-2)})), where phi is the golden ratio. - _Vladimir Shevelev_, Jun 24 2010, Jun 27 2010
   - For n>=1, a(n+1)=ceil(phi*a(n)), if n is even and a(n+1)=floor(phi*a(n)), if n is odd (phi = golden ratio). - _Vladimir Shevelev_, Jul 01 2010
   - a(n) = 2*a(n-2) + a(n-3), n>2. - _Gary Detlefs_, Sep 08 2010
   - a(2^n) = Prod_{i=0..n-1} <a href="http://oeis.org/A000032">A000032</a>(2^i). - _Vladimir Shevelev_, Nov 28 2010
   - a(n)^2 - a(n-1)^2 = a(n+1)*a(n-2), see <a href="http://oeis.org/A121646">A121646</a>.
   - a(n) = sqrt((-1)^k*(a(n+k)^2 - a(k)*a(2n+k))), for any k. - _Gary Detlefs_, Dec 03 2010
   - F(2*n) = F(n+2)^2 - F(n+1)^2 - 2*F(n)^2. - _Richard R. Forberg_, Jun 04 2011
   - (-1)^(n+1) = F(n)^2 + F(n)*F(1+n) - F(1+n)^2.
   -   F(n) = -F(n+2)(-2 + (F(n+1))^4 + 2*(F(n+1)^3*F(n+2)) - (F(n+1)*F(n+2))^2 2*F(n+1)(F(n+2))^3 + (F(n+2))^4)- F(n+1). - _Artur Jasinski_, Nov 17 2011
   - F(n) = 1 + Sum_{x=1..n-2} F(x). - _Joseph P. Shoulak_, Feb 05 2012
   - F(n) = 4*F(n-2) - 2*F(n-3) - F(n-6). - _Gary Detlefs_, Apr 01 2012
   - F(n) = round(phi^(n+1)/(phi+2)). - _Thomas Ordowski_, Apr 20 2012
   - From _Sergei N. Gladkovskii_, Jun 03 2012: 
       - G.f. A(x) = x/(1-x-x^2) = G(0)/sqrt(5) where G(k)= 1 -((-1)^k)*2^k/(a^k - b*x*a^k*2^k/(b*x*2^k - 2*((-1)^k)*c^k/G(k+1))) and a=3+sqrt(5), b=1+sqrt(5), c=3-sqrt(5); (continued fraction, 3rd kind, 3-step).
       - Let E(x) be the e.g.f., i.e.,
       - E(x) = 1*x + 1/2*x^2 + 1/3*x^3 + 1/8*x^4 + 1/24*x^5 + 1/90*x^6 + 13/5040*x^7 + ...; then
       - E(x) = G(0)/sqrt(5); G(k)= 1 -((-1)^k)*2^k/(a^k - b*x*a^k*2^k/(b*x*2^k - 2*((-1)^k)*(k+1)*c^k/G(k+1))), where a=3+sqrt(5), b=1+sqrt(5), c=3-sqrt(5); (continued fraction, 3rd kind, 3-step).
   - From _Hieronymus Fischer_, Nov 30 2012: 
       - Fib(n) = 1 + Sum_{j_1=1..n-2} 1 + Sum_{j_1=1..n-2} Sum_{j_2=1..j_1-2} 1 + Sum_{j_1=1..n-2} Sum_{j_2=1..j_1-2} Sum_{j_3=1..j_2-2} 1 + ... + Sum_{j_1=1..n-2} Sum_{j_2=1..j_1-2} Sum_{j_3=1..j_2-2} ... Sum_{j_k=1..j_(k-1)-2} 1, where k = floor((n-1)/2).
       - Example: Fib(6) = 1 + Sum_{j=1..4} 1 + Sum_{j=1..4} Sum_{k=1..(j-2)} 1 + 0 = 1 + (1 + 1 + 1 + 1) + (1 + (1 + 1)) = 8.
       - Fib(n) = Sum_{j=0..k} S(j+1,n-2j), where k = floor((n-1)/2) and the S(j,n) are the n-th j-simplex sums: S(1,n) = 1 is the 1-simplex sum, S(2,n) = Sum_{k=1..n} S(1,k) = 1+1+...+1 = n is the 2-simplex sum, S(3,n) = Sum_{k=1..n} S(2,k) = 1+2+3+...+n is the 3-simplex sum (= triangular numbers = <a href="http://oeis.org/A000217">A000217</a>), S(4,n) = Sum_{k=1..n} S(3,k) = 1+3+6+...+n(n+1)/2 is the 4-simplex sum (= tetrahedral numbers = <a href="http://oeis.org/A000292">A000292</a>) and so on.
       - Since S(j,n) = binomial(n-2+j,j-1), the formula above equals the well-known binomial formula, essentially. 
   - G.f. A(x) = x / (1 - x / (1 - x / (1 + x))). - _Michael Somos_, Jan 04 2013
   - Sum_{n>=1} (-1)^(n-1)/(a(n)*a(n+1)) = 1/phi (phi=golden ratio). - _Vladimir Shevelev_, Feb 22 2013
   - From _Vladimir Shevelev_, Feb 24 2013: 
       - (1) Expression a(n+1) via a(n): a(n+1) = (a(n) + sqrt(5*(a(n))^2 + 4*(-1)^n))/2;
       - (2) Sum_{k=1...n} (-1)^(k-1)/(a(k)*a(k+1)) = a(n)/a(n+1);
       - (3) a(n)/a(n+1) = 1/phi + r(n), where |r(n)| < 1/(a(n+1)*a(n+2)). 
   - F(n+1) = F(n)/2 + sqrt((-1)^n + 5*F(n)^2/4), n>=0. F(n+1) = U_n(i/2)/i^n, (U:= Chebyshef 2nd kind). - _Bill Gosper_, Mar 04 2013
   - G.f.: -Q(0) where Q(k) = 1 - (1+x)/(1 - x/(x - 1/Q(k+1) )); (continued fraction). - _Sergei N. Gladkovskii_, Mar 06 2013
   - G.f.: x-1-1/x + 1/x/Q(0), where Q(k) = 1 - (k+1)*x/(1 - x/(x - (k+1)/Q(k+1))); (continued fraction). - _Sergei N. Gladkovskii_, Apr 23 2013
   - G.f.: x*G(0), where G(k)= 1 + x*(1+x)/(1 - x*(1+x)/(x*(1+x) + 1/G(k+1) )); (continued fraction). - _Sergei N. Gladkovskii_, Jul 08 2013
   - G.f.: x^2 - 1 + 2*x^2/(W(0)-2), where W(k) = 1 + 1/(1 - x*(k + x)/( x*(k+1 + x) + 1/W(k+1) )); (continued fraction). - _Sergei N. Gladkovskii_, Aug 28 2013
   - G.f.: Q(0) -1, where Q(k) = 1 + x^2 + (k+2)*x -x*(k+1 + x)/Q(k+1); (continued fraction). - _Sergei N. Gladkovskii_, Oct 06 2013
   - Let b(n) = b(n-1) + b(n-2), with b(0) = 0, b(1) = phi. Then, for n>=2, F(n)= floor(b(n-1)) if n is even, F(n) = ceil(b(n-1)), if n is odd, with convergence. - _Richard R. Forberg_, Jan 19 2014
   - a(n) = Sum_{t1*g(1)+t2*g(2)+...+tn*g(n)=n} multinomial(t1+t2 +...+tn,t1,t2,...,tn), where g(k)=2*k-1. - _Mircea Merca_, Feb 27 2014
   - F(n) = round(sqrt(F(n-1)^2 + F(n)^2 + F(n+1)^2)/2), for n > 0. This rule appears to apply to any sequence of the form a(n) = a(n-1) + a(n-2), for any two values of a(0) and a(1), if n is sufficiently large. - _Richard R. Forberg_, Jul 27 2014
   - F(n) = round(2/(1/F(n) + 1/F(n+1) + 1/F(n+2)), for n > 0. This rule also appears to apply to any sequence of the form a(n) = a(n-1) + a(n-2), for any two values of a(0) and a(1), if n is sufficiently large. - _Richard R. Forberg_, Aug 03 2014
   - F(n) = round(1/(Sum_{j>=n+2} 1/F(j))). - _Richard R. Forberg_, Aug 14 2014
   - a(n) = hypergeometric([-n/2+1/2, -n/2+1], [-n+1], -4) for n>=2. - _Peter Luschny_, Sep 19 2014
   - F(n) = (L(n+1)^2 - L(n-1)^2)/(5*L(n)), where L(n) is <a href="http://oeis.org/A000032">A000032</a>(n), with a similar inverse relationship. - _Richard R. Forberg_, Nov 17 2014
   - Consider the graph G[1-vertex;1-loop,2-loop] in comment above. Construct the power matrix array T(n,j)=[A^*j]*[S^*(j-1)] where A=(1,1,0,...) and S=(0,1,0,...)(<a href="http://oeis.org/A063524">A063524</a>). [* is convolution operation] Define S^*0=I with I=(1,0,...). Then T(n,j) counts n-walks containing (j) loops and a(n-1) = Sum_{j=1...n} T(n,j). - _David Neil McGrath_, Nov 21 2014
   - Define F(-n) to be F(n) for n odd and -F(n) for n even. Then for all n and k, F(n) = F(k)*F(n-k+3) - F(k-1)*F(n-k+2) - F(k-2)*F(n-k) + (-1)^k*F(n-2k+2). - _Charlie Marion_, Dec 04 2014
   - F(n+k)^2 - L(k)*F(n)*F(n+k) + (-1)^k*F(n)^2 = (-1)^n*F(k)^2, if L(k) = <a href="http://oeis.org/A000032">A000032</a>(k). - _Alexander Samokrutov_, Jul 20 2015
   - F(2*n) = F(n+1)^2 - F(n-1)^2, similar to Koshy (D) and Forberg 2011, but different. - _Hermann Stamm-Wilbrandt_, Aug 12 2015
   - F(n+1) = ceiling( (1/phi)*Sum_{k=0..n} F(k) ). - _Tom Edgar_, Sep 10 2015
   - a(n) = (L(n-3) + L(n+3))/10 where L(n)=<a href="http://oeis.org/A000032">A000032</a>(n). - _J. M. Bergot_, Nov 25 2015
   - From _Bob Selcoe_, Mar 27 2016 :
       - F(n) = (F(2n+k+1) - F(n+1)*F(n+k+1))/F(n+k), k>=0.
       - Thus when k=0: F(n) = sqrt(F(2n+1) - F(n+1)^2).
       - F(n) = cbrt(F(3n) - F(n+1)^3 + F(n-1)^3).
       - F(n+2k) = binomial transform of any subsequence starting with F(n). Example F(6)=8: 1*8 = F(6)=8; 1*8 + 1*13 = F(8)=21; 1*8 + 2*13 + 1*21 = F(10)=55; 1*8 + 3*13 + 3*21 + 1*34 = F(12)=144, etc. This formula applies to Fibonacci-type sequences with any two seed values for a(0) and a(1) (e.g., Lucas sequence <a href="http://oeis.org/A000032">A000032</a>: a(0)=2, a(1)=1).
   - F(n) = L(k)*F(n-k) + (-1)^(k+1)*F(n-2k) for all k>=0, where L(k) = <a href="http://oeis.org/A000032">A000032</a>(k). - _Anton Zakharov_, Aug 02 2016
   - From _Ilya Gutkovskiy_, Aug 03 2016: 
       - a(n) = F_n(1), where F_n(x) are the Fibonacci polynomials.
       - Inverse binomial transform of <a href="http://oeis.org/A001906">A001906</a>.
       - Number of zeros in substitution system {0 -> 11, 1 -> 1010} at step n from initial string "1" (1 -> 1010 -> 101011101011 -> ...) multiplied by 1/<a href="http://oeis.org/A000079">A000079</a>(n). 

_Cross references_:
   - Cf. <a href="http://oeis.org/A039834">A039834</a> (signed Fibonacci numbers), <a href="http://oeis.org/A001690">A001690</a> (complement), <a href="http://oeis.org/A000213">A000213</a>, <a href="http://oeis.org/A000288">A000288</a>, <a href="http://oeis.org/A000322">A000322</a>, <a href="http://oeis.org/A000383">A000383</a>, <a href="http://oeis.org/A060455">A060455</a>, <a href="http://oeis.org/A030186">A030186</a>, <a href="http://oeis.org/A020695">A020695</a>, <a href="http://oeis.org/A020701">A020701</a>, <a href="http://oeis.org/A071679">A071679</a>, <a href="http://oeis.org/A099731">A099731</a>, <a href="http://oeis.org/A100492">A100492</a>, <a href="http://oeis.org/A094216">A094216</a>, <a href="http://oeis.org/A094638">A094638</a>, <a href="http://oeis.org/A000108">A000108</a>, <a href="http://oeis.org/A101399">A101399</a>, <a href="http://oeis.org/A101400">A101400</a>, <a href="http://oeis.org/A001611">A001611</a>, <a href="http://oeis.org/A000071">A000071</a>, <a href="http://oeis.org/A157725">A157725</a>, <a href="http://oeis.org/A001911">A001911</a>, <a href="http://oeis.org/A157726">A157726</a>, <a href="http://oeis.org/A006327">A006327</a>, <a href="http://oeis.org/A157727">A157727</a>, <a href="http://oeis.org/A157728">A157728</a>, <a href="http://oeis.org/A157729">A157729</a>, <a href="http://oeis.org/A167616">A167616</a>, <a href="http://oeis.org/A059929">A059929</a>, <a href="http://oeis.org/A144152">A144152</a>, <a href="http://oeis.org/A152063">A152063</a>, <a href="http://oeis.org/A114690">A114690</a>, <a href="http://oeis.org/A003893">A003893</a>, <a href="http://oeis.org/A000032">A000032</a>, <a href="http://oeis.org/A060441">A060441</a>, <a href="http://oeis.org/A000930">A000930</a>, <a href="http://oeis.org/A003269">A003269</a>, <a href="http://oeis.org/A000957">A000957</a>, <a href="http://oeis.org/A057078">A057078</a>, <a href="http://oeis.org/A007317">A007317</a>, <a href="http://oeis.org/A091867">A091867</a>, <a href="http://oeis.org/A104597">A104597</a>, <a href="http://oeis.org/A249548">A249548</a>, <a href="http://oeis.org/A262342">A262342</a>, <a href="http://oeis.org/A001060">A001060</a>, <a href="http://oeis.org/A022095">A022095</a>.
   - First row of arrays <a href="http://oeis.org/A103323">A103323</a>, <a href="http://oeis.org/A234357">A234357</a>. Second row of arrays <a href="http://oeis.org/A099390">A099390</a>, <a href="http://oeis.org/A048887">A048887</a>, and <a href="http://oeis.org/A092921">A092921</a> (k-generalized Fibonacci numbers).
   - a(n) = <a href="http://oeis.org/A094718">A094718</a>(4, n). a(n) = <a href="http://oeis.org/A101220">A101220</a>(0, j, n).
   - a(n) = <a href="http://oeis.org/A090888">A090888</a>(0, n+1) = <a href="http://oeis.org/A118654">A118654</a>(0, n+1) = <a href="http://oeis.org/A118654">A118654</a>(1, n-1) = <a href="http://oeis.org/A109754">A109754</a>(0, n) = <a href="http://oeis.org/A109754">A109754</a>(1, n-1), for n > 0.
   - Fibonacci-Pascal triangles: <a href="http://oeis.org/A027926">A027926</a>, <a href="http://oeis.org/A036355">A036355</a>, <a href="http://oeis.org/A037027">A037027</a>, <a href="http://oeis.org/A074829">A074829</a>, <a href="http://oeis.org/A105809">A105809</a>, <a href="http://oeis.org/A109906">A109906</a>, <a href="http://oeis.org/A111006">A111006</a>, <a href="http://oeis.org/A114197">A114197</a>, <a href="http://oeis.org/A162741">A162741</a>, <a href="http://oeis.org/A228074">A228074</a>.
   - Boustrophedon transforms: <a href="http://oeis.org/A000738">A000738</a>, <a href="http://oeis.org/A000744">A000744</a>.
   - Powers: <a href="http://oeis.org/A103323">A103323</a>, <a href="http://oeis.org/A105317">A105317</a>, <a href="http://oeis.org/A254719">A254719</a>.
   - Numbers of prime factors: <a href="http://oeis.org/A022307">A022307</a> and <a href="http://oeis.org/A038575">A038575</a>.
   - Cf. <a href="http://oeis.org/A163733">A163733</a>.



### Search by sequence of numbers

In [56]:
searchable = search(seq=[1,1,2,5,14,42, 132, 429], max_results=4)

●

In [57]:
searchable(comment=lambda i,c: i in range(10))

_Results for query: <a href='https://oeis.org/search?fmt=json&start=0&q=1%2C+1%2C+2%2C+5%2C+14%2C+42%2C+132%2C+429'>https://oeis.org/search?fmt=json&start=0&q=1%2C+1%2C+2%2C+5%2C+14%2C+42%2C+132%2C+429</a>_<br><hr><div align='center'><b><a href='http://oeis.org/A000108'>A000108</a></b>: Catalan numbers: C(n) = binomial(2n,n)/(n+1) = (2n)!/(n!(n+1)!). Also called Segner numbers.<br></div>

by _N. J. A. Sloane_

_Keywords_: `core,nonn,easy,eigen,nice`

_Data_:

$$
\begin{array}{c|ccccccccccccccc }
    n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
    \hline
    A000108(n) & 1 & 1 & 2 & 5 & 14 & 42 & 132 & 429 & 1430 & 4862 & 16796 & 58786 & 208012 & 742900 & 2674440
\end{array}
$$
        

_Comments_:
   - The solution to Schröder's first problem. A very large number of combinatorial interpretations are known - see references, esp. Stanley, Enumerative Combinatorics, Volume 2. This is probably the longest entry in the OEIS, and rightly so.
   - Number of ways to insert n pairs of parentheses in a word of n+1 letters. E.g., for n=2 there are 2 ways: ((ab)c) or (a(bc)); for n=3 there are 5 ways: ((ab)(cd)), (((ab)c)d), ((a(bc))d), (a((bc)d)), (a(b(cd))).
   - Consider all the binomial(2n,n) paths on squared paper that (i) start at (0, 0), (ii) end at (2n, 0) and (iii) at each step, either make a (+1,+1) step or a (+1,-1) step. Then the number of such paths that never go below the x-axis (Dyck paths) is C(n). [Chung-Feller]
   - Number of noncrossing partitions of the n-set. For example, of the 15 set partitions of the 4-set, only [{13},{24}] is crossing, so there are a(4)=14 noncrossing partitions of 4 elements. - _Joerg Arndt_, Jul 11 2011
   - a(n-1) is the number of ways of expressing an n-cycle in the symmetric group S_n as a product of n-1 transpositions (u_1,v_1)*(u_2,v_2)*...*(u_{n-1},v_{n-1}) where u_k<=u_j and v_k<=v_j for k<j; see example.  If the condition is dropped, one obtains <a href="http://oeis.org/A000272">A000272</a>. - _Joerg Arndt_ and Greg Stevenson, Jul 11 2011
   - a(n) is the number of ordered rooted trees with n nodes, not including the root. See the Conway-Guy reference where these rooted ordered trees are called plane bushes. See also the Bergeron et al. reference, Example 4, p. 167. - _Wolfdieter Lang_, Aug 07 2007
   - As shown in the paper from Beineke and Pippert (1971), a(n-2)=D(n) is the number of labeled dissections of a disk, related to the number R(n)=<a href="http://oeis.org/A001761">A001761</a>(n-2) of labeled planar 2-trees having n vertices and rooted at a given exterior edge, by the formula D(n)=R(n)/(n-2)!. - _M. F. Hasler_, Feb 22 2012
   - Shifts one place left when convolved with itself.
   - For n >= 1 a(n) is also the number of rooted bicolored unicellular maps of genus 0 on n edges. - Ahmed Fares (ahmedfares(AT)my-deja.com), Aug 15 2001
   - Ways of joining 2n points on a circle to form n nonintersecting chords. (If no such restriction imposed, then ways of forming n chords is given by (2n-1)!!=(2n)!/n!2^n=<a href="http://oeis.org/A001147">A001147</a>(n).)

_Formulae_:
   - a(n) = <a href="http://oeis.org/A000984">A000984</a>(n)/(n+1) = binomial(2*n, n)/(n+1) = (2*n)!/(n!*(n+1)!).
   - a(n) = binomial(2*n, n)-binomial(2*n, n-1).
   - a(n) = Sum_{k=0..n-1} a(k)a(n-1-k).
   - a(n) = Product_{k=2..n} (1 + n/k), if n>1.
   - G.f.: A(x) = (1 - sqrt(1 - 4*x)) / (2*x). G.f. A(x) satisfies A = 1 + x*A^2.
   - a(n+1) = Sum_{i} binomial(n, 2*i)*2^(n-2*i)*a(i). - Touchard
   - 2*(2*n-1)*a(n-1) = (n+1)*a(n).
   - It is known that a(n) is odd if and only if n=2^k-1, k=0, 1, 2, 3, ... - _Emeric Deutsch_, Aug 04 2002, corrected by _M. F. Hasler_, Nov 08 2015
   - Using the Stirling approximation in <a href="http://oeis.org/A000142">A000142</a> we get the asymptotic expansion a(n) ~ 4^n / (sqrt(Pi * n) * (n + 1)). - Dan Fux (dan.fux(AT)OpenGaia.com or danfux(AT)OpenGaia.com), Apr 13 2001
   - Integral representation: a(n) = int(x^n*sqrt((4-x)/x), x=0..4)/(2*Pi). - _Karol A. Penson_, Apr 12 2001
   - E.g.f.: exp(2*x)*(I_0(2*x)-I_1(2*x)), where I_n is Bessel function. - _Karol A. Penson_, Oct 07 2001
   - Polygorial(n, 6)/Polygorial(n, 3). - Daniel Dockery (peritus(AT)gmail.com), Jun 24 2003
   - G.f. A(x) satisfies ((A(x) + A(-x)) / 2)^2 = A(4*x^2). - _Michael Somos_, Jun 27, 2003
   - G.f. A(x) satisfies Sum_{k>=1} k(A(x)-1)^k = Sum_{n >= 1} 4^{n-1}*x^n. - Shapiro, Woan, Getu
   - a(n+m) = Sum_{k} <a href="http://oeis.org/A039599">A039599</a>(n, k)*<a href="http://oeis.org/A039599">A039599</a>(m, k). - _Philippe Deléham_, Dec 22 2003
   - a(n+1) = (1/(n+1))*Sum_{k=0..n} a(n-k)*binomial(2k+1, k+1). - _Philippe Deléham_, Jan 24 2004
   - a(n) = Sum_{k>=0} <a href="http://oeis.org/A008313">A008313</a>(n, k)^2. - _Philippe Deléham_, Feb 14 2004
   - a(m+n+1) = Sum_{k>=0} <a href="http://oeis.org/A039598">A039598</a>(m, k)*<a href="http://oeis.org/A039598">A039598</a>(n, k). - _Philippe Deléham_, Feb 15 2004
   - C(n-1) = binomial(2*n-2,n-1)/n = (1/n!) * [ n^(n-1) + ( binomial(n-2,1) + binomial(n-2,2) )*n^(n-2) + ( 2*binomial(n-3,1) + 7*binomial(n-3,2) + 8*binomial(n-3,3) + 3*binomial(n-3,4) )*n^(n-3) + ( 6*binomial(n-4,1) + 38*binomial(n-4,2) + 93*binomial(n-4,3) + 111*binomial(n-4,4) + 65*binomial(n-4,5) + 15*binomial(n-4,6) )*n^(n-4) + ... ]. - _André F. Labossière_, Nov 10 2004, corrected by _M. F. Hasler_, Nov 10 2015
   - a(n) = Sum_{k=0..n} (-1)^k*2^(n-k)*binomial(n, k)*binomial(k, floor(k/2)). - Paul Barry, Jan 27 2005
   - Sum_{n>=0} 1/a(n) = 2 + 4*Pi/3^(5/2) = F(1,2;1/2;1/4) = 2.806133050770763... (see L'Univers de Pi link). - _Gerald McGarvey_ and _Benoit Cloitre_, Feb 13 2005
   - a(n) = Sum_{k=0..floor(n/2)} ((n-2*k+1)*binomial(n, n-k)/(n-k+1))^2, which is equivalent to: a(n) = Sum_{k=0..n} <a href="http://oeis.org/A053121">A053121</a>(n, k)^2, for n>=0. - _Paul D. Hanna_, Apr 23 2005
   - a((m+n)/2) = Sum_{k>=0} <a href="http://oeis.org/A053121">A053121</a>(m, k)*<a href="http://oeis.org/A053121">A053121</a>(n, k) if m+n is even. - _Philippe Deléham_, May 26 2005
   - E.g.f. Sum_{n>=0} a(n) * x^(2*n) / (2*n)! = BesselI(1, 2*x) / x. - _Michael Somos_, Jun 22 2005
   - Given g.f. A(x), then B(x) = x * A(x^3) satisfies 0 = f(x, B(X)) where f(u, v) = u - v + (u*v)^2 or B(x) = x + (x * B(x))^2 which implies B(-B(x)) = -x and also (1 + B^3) / B^2 = (1 - x^3) / x^2. - _Michael Somos_, Jun 27 2005
   - a(n) = a(n-1)*(4-6/(n+1)). a(n) = 2a(n-1)*(8a(n-2)+a(n-1))/(10a(n-2)-a(n-1)). - _Franklin T. Adams-Watters_, Feb 08 2006
   - Sum_{k>=1} a(k)/4^k = 1. - _Franklin T. Adams-Watters_, Jun 28 2006
   - a(n) = <a href="http://oeis.org/A047996">A047996</a>(2*n+1, n). - _Philippe Deléham_, Jul 25 2006
   - Binomial transform of <a href="http://oeis.org/A005043">A005043</a>. - _Philippe Deléham_, Oct 20 2006
   - a(n) = Sum_{k=0..n} (-1)^k*<a href="http://oeis.org/A116395">A116395</a>(n,k). - _Philippe Deléham_, Nov 07 2006
   - a(n) = (1/(s-n))*Sum_{k=0..n} (-1)^k (k+s-n)*binomial(s-n,k) * binomial(s+n-k,s) with s a nonnegative free integer [H. W. Gould].
   - a(k) = Sum_{i=1..k} |<a href="http://oeis.org/A008276">A008276</a>(i,k)| * (k-1)^(k-i) / k!. - _André F. Labossière_, May 29 2007
   - a(n) = Sum_{k=0..n} <a href="http://oeis.org/A129818">A129818</a>(n,k) * <a href="http://oeis.org/A007852">A007852</a>(k+1). - _Philippe Deléham_, Jun 20 2007
   - a(n) = Sum_{k=0..n} <a href="http://oeis.org/A109466">A109466</a>(n,k) * <a href="http://oeis.org/A127632">A127632</a>(k). - _Philippe Deléham_, Jun 20 2007
   - Row sums of triangle <a href="http://oeis.org/A124926">A124926</a>. - _Gary W. Adamson_, Oct 22 2007
   - Lim_{n->infinity}(1+Sum_{k=0..n}a(k)/<a href="http://oeis.org/A004171">A004171</a>(k)) = 4/Pi. - _Reinhard Zumkeller_, Aug 26 2008
   - a(n) = Sum_{k=0..n} <a href="http://oeis.org/A120730">A120730</a>(n,k)^2 and a(k+1) = Sum_{n>=k} <a href="http://oeis.org/A120730">A120730</a>(n,k). - _Philippe Deléham_, Oct 18 2008
   - Given an integer t >= 1 and initial values u = [a_0, a_1, ..., a_{t-1}], we may define an infinite sequence Phi(u) by setting a_n = a_{n-1} + a_0*a_{n-1} + a_1*a_{n-2} + ... + a_{n-2}*a_1 for n >= t. For example, the present sequence is Phi([1]) (also Phi([1,1])). - _Gary W. Adamson_, Oct 27 2008
   - a(n) = Sum_{l_1=0..n+1} Sum_{l_2=0..n}...Sum_{l_i=0..n-i}...Sum_{l_n=0..1} delta(l_1,l_2,...,l_i,...,l_n) where delta(l_1,l_2,...,l_i,...,l_n) = 0 if any l_i < l_(i+1) and l_(i+1) <> 0 for i=1..n-1 and delta(l_1,l_2,...,l_i,...,l_n) = 1 otherwise. - _Thomas Wieder_, Feb 25 2009
   - Let A(x) be the g.f., then B(x)=x*A(x) satisfies the differential equation B'(x)-2*B'(x)*B(x)-1=0. - _Vladimir Kruchinin_, Jan 18 2011
   - G.f.: 1/(1-x/(1-x/(1-x/(...)))) (continued fraction). - _Joerg Arndt_, Mar 18 2011
   - With F(x) = (1-2*x-sqrt(1-4*x))/(2*x) an o.g.f. in x for the Catalan series, G(x) = x/(1+x)^2 is the compositional inverse of F (nulling the n=0 term). - _Tom Copeland_, Sep 04 2011
   - With H(x) = 1/(dG(x)/dx) = (1+x)^3 / (1-x), the n-th Catalan number is given by (1/n!)*((H(x)*d/dx)^n)x evaluated at x=0, i.e., F(x) = exp(x*H(u)*d/du)u, evaluated at u = 0. Also, dF(x)/dx = H(F(x)), and H(x) is the o.g.f. for <a href="http://oeis.org/A115291">A115291</a>. - _Tom Copeland_, Sep 04 2011
   - With F(x) = {1-sqrt[1-4*x]}/2 an o.g.f. in x for the Catalan series, G(x)= x*(1-x) is the compositional inverse and this relates the Catalan numbers to the row sums of <a href="http://oeis.org/A125181">A125181</a>. - _Tom Copeland_, Sep 30 2011
   - With H(x)=1/(dG(x)/dx)= 1/(1-2x), the n-th Catalan number (offset 1) is given by (1/n!)*((H(x)*d/dx)^n)x evaluated at x=0, i.e., F(x) = exp(x*H(u)*d/du)u, evaluated at u = 0. Also, dF(x)/dx = H(F(x)). - _Tom Copeland_, Sep 30 2011
   - G.f.: (1-sqrt(1-4*x))/(2*x)=G(0) where G(k)=1+(4*k+1)*x/(k+1-2*x*(k+1)*(4*k+3)/(2*x*(4*k+3)+(2*k+3)/G(k+1))); (continued fraction). - _Sergei N. Gladkovskii_, Nov 30 2011
   - E.g.f.: exp(2*x)*(BesselI(0,2*x)-BesselI(1,2*x))=G(0) where G(k)=1+(4*k+1)*x/((k+1)*(2*k+1)-x*(k+1)*(2*k+1)*(4*k+3)/(x*(4*k+3)+(k+1)*(2*k+3)/G(k+1))); (continued fraction). - _Sergei N. Gladkovskii_, Nov 30 2011
   - E.g.f.: Hypergeometric([1/2],[2],4*x) which coincides with the e.g.f. given just above, and also by _Karol A. Penson_ further above. - _Wolfdieter Lang_, Jan 13 2012
   - a(n) = <a href="http://oeis.org/A208355">A208355</a>(2*n-1) = <a href="http://oeis.org/A208355">A208355</a>(2*n) for n > 0. - _Reinhard Zumkeller_, Mar 04 2012
   - G.f.: 1 + 2*x/(U(0)-2*x) where U(k)= k*(4*x+1) + 2*x + 2 - x*(2*k+3)*(2*k+4)/U(k+1); (continued fraction, Euler's 1st kind, 1-step). - _Sergei N. Gladkovskii_, Sep 20 2012
   - G.f.: hypergeom([1/2,1],[2],4*x). - _Joerg Arndt_, Apr 06 2013
   - Special values of Jacobi polynomials, in Maple notation: a(n) = 4^n*JacobiP(n,1,-1/2-n,-1)/(n+1). - _Karol A. Penson_, Jul 28 2013
   - For n > 0: a(n) = sum of row n in triangle <a href="http://oeis.org/A001263">A001263</a>. - _Reinhard Zumkeller_, Oct 10 2013
   - a(n) = binomial(2n,n-1)/n and a(n) mod n = binomial(2n,n) mod n = <a href="http://oeis.org/A059288">A059288</a>(n). - _Jonathan Sondow_, Dec 14 2013
   - a(n-1) = sum(t1+2*t2+...+n*tn=n, (-1)^(1+t1+t2+...+tn)*multinomial(t1+t2 +...+tn,t1,t2,...,tn)*a(1)^t1*a(2)^t2*...*a(n)^tn). - _Mircea Merca_, Feb 27 2014
   - a(n) = sum_{k=1..n} binomial(n+k-1,n)/n if n>0. _Alexander Adamchuk_, Mar 25 2014
   - a(n) = -2^(2*n+1) * binomial(n-1/2, -3/2). - _Peter Luschny_, May 06 2014
   - a(n) = (4*<a href="http://oeis.org/A000984">A000984</a>(n)-<a href="http://oeis.org/A000984">A000984</a>(n+1))/2. - _Stanislav Sykora_, Aug 09 2014
   - a(n) = <a href="http://oeis.org/A246458">A246458</a>(n) * <a href="http://oeis.org/A246466">A246466</a>(n). - _Tom Edgar_, Sep 02 2014
   - a(n) = (2*n)!*[x^(2*n)]hypergeom([],[2],x^2). - _Peter Luschny_, Jan 31 2015
   - a(n) = 4^(n-1)*hypergeom([3/2, 1-n], [3], 1). - _Peter Luschny_, Feb 03 2015
   - a(2n) = 2*<a href="http://oeis.org/A000150">A000150</a>(2n); a(2n+1) = 2*<a href="http://oeis.org/A000150">A000150</a>(2n+1) + a(n). - _John Bodeen_, Jun 24 2015
   - a(n) = Sum_{t=1, n+1} n^(t-1)*abs(stirling1(n+1, t)) / Sum_{t=1, n+1} abs(stirling1(n+1, t)), for n >0, see (10) in Cereceda link. - _Michel Marcus_, Oct 06 2015
   - a(n) ~ 4^(n-2)*(128+160/N^2+84/N^4+715/N^6-10180/N^8)/(N^(3/2)*Pi^(1/2)) where N=4*n+3. - _Peter Luschny_, Oct 14 2015
   - a(n) = sum_{k=1..floor((n+1)/2)} (-1)^(k-1)*binomial(n+1-k,k)*a(n-k) if n > 0; and a(0) = 1. - _David Pasino_, Jun 29 2016
   - Sum_{n>=0} (-1)^n/a(n) = 14/25 - 24*arccsch(2)/(25*sqrt(5)) = 14/25 - 24*<a href="http://oeis.org/A002390">A002390</a>/(25*sqrt(5)) = 0.353403708337278061333... - _Ilya Gutkovskiy_, Jun 30 2016

_Cross references_:
   - Cf. <a href="http://oeis.org/A000142">A000142</a>, <a href="http://oeis.org/A000245">A000245</a>, <a href="http://oeis.org/A000344">A000344</a>, <a href="http://oeis.org/A000588">A000588</a>, <a href="http://oeis.org/A000957">A000957</a>, <a href="http://oeis.org/A000984">A000984</a>, <a href="http://oeis.org/A001392">A001392</a>, <a href="http://oeis.org/A001453">A001453</a>, <a href="http://oeis.org/A001791">A001791</a>, <a href="http://oeis.org/A002057">A002057</a>, <a href="http://oeis.org/A002420">A002420</a>, <a href="http://oeis.org/A003046">A003046</a>, <a href="http://oeis.org/A003517">A003517</a>, <a href="http://oeis.org/A003518">A003518</a>, <a href="http://oeis.org/A003519">A003519</a>, <a href="http://oeis.org/A006480">A006480</a>, <a href="http://oeis.org/A008276">A008276</a>, <a href="http://oeis.org/A008549">A008549</a>, <a href="http://oeis.org/A014137">A014137</a>, <a href="http://oeis.org/A014138">A014138</a>, <a href="http://oeis.org/A014140">A014140</a>, <a href="http://oeis.org/A022553">A022553</a>, <a href="http://oeis.org/A024492">A024492</a>, <a href="http://oeis.org/A032357">A032357</a>, <a href="http://oeis.org/A032443">A032443</a>, <a href="http://oeis.org/A039599">A039599</a>, <a href="http://oeis.org/A048990">A048990</a>, <a href="http://oeis.org/A059288">A059288</a>, <a href="http://oeis.org/A068875">A068875</a>, <a href="http://oeis.org/A069640">A069640</a>, <a href="http://oeis.org/A086117">A086117</a>, <a href="http://oeis.org/A094216">A094216</a>, <a href="http://oeis.org/A094638">A094638</a>, <a href="http://oeis.org/A094639">A094639</a>, <a href="http://oeis.org/A098597">A098597</a>, <a href="http://oeis.org/A099731">A099731</a>, <a href="http://oeis.org/A119822">A119822</a>, <a href="http://oeis.org/A120304">A120304</a>, <a href="http://oeis.org/A124926">A124926</a>, <a href="http://oeis.org/A129763">A129763</a>, <a href="http://oeis.org/A137697">A137697</a>, <a href="http://oeis.org/A154559">A154559</a>, <a href="http://oeis.org/A161581">A161581</a>, <a href="http://oeis.org/A167892">A167892</a>, <a href="http://oeis.org/A167893">A167893</a>, <a href="http://oeis.org/A179277">A179277</a>, <a href="http://oeis.org/A211611">A211611</a>.
   - A row of <a href="http://oeis.org/A060854">A060854</a>.
   - See <a href="http://oeis.org/A001003">A001003</a>, <a href="http://oeis.org/A001190">A001190</a>, <a href="http://oeis.org/A001699">A001699</a>, <a href="http://oeis.org/A000081">A000081</a> for other ways to count parentheses.
   - Enumerates objects encoded by <a href="http://oeis.org/A014486">A014486</a>.
   - A diagonal of any of the essentially equivalent arrays <a href="http://oeis.org/A009766">A009766</a>, <a href="http://oeis.org/A030237">A030237</a>, <a href="http://oeis.org/A033184">A033184</a>, <a href="http://oeis.org/A059365">A059365</a>, <a href="http://oeis.org/A099039">A099039</a>, <a href="http://oeis.org/A106566">A106566</a>, <a href="http://oeis.org/A130020">A130020</a>, <a href="http://oeis.org/A047072">A047072</a>.
   - Cf. <a href="http://oeis.org/A051168">A051168</a> (diagonal of the square array described).
   - Cf. <a href="http://oeis.org/A033552">A033552</a>, <a href="http://oeis.org/A176137">A176137</a> (partitions into Catalan numbers).
   - Cf. <a href="http://oeis.org/A000753">A000753</a>, <a href="http://oeis.org/A000736">A000736</a> (Boustrophedon transforms).
   - Cf. <a href="http://oeis.org/A120303">A120303</a> (largest prime factor of Catalan number).
   - Cf. <a href="http://oeis.org/A121839">A121839</a> (reciprocal Catalan constant).
   - Cf. <a href="http://oeis.org/A038003">A038003</a>, <a href="http://oeis.org/A119861">A119861</a>, <a href="http://oeis.org/A119908">A119908</a>, <a href="http://oeis.org/A120274">A120274</a>, <a href="http://oeis.org/A120275">A120275</a> (odd Catalan number).
   - Cf. <a href="http://oeis.org/A002390">A002390</a> (decimal expansion of natural logarithm of golden ratio).
   - Coefficients of square root of the g.f. are <a href="http://oeis.org/A001795">A001795</a>/<a href="http://oeis.org/A046161">A046161</a>.
   - For a(n) mod 6 see <a href="http://oeis.org/A259667">A259667</a>.
   - For a(n) in base 2 see <a href="http://oeis.org/A264663">A264663</a>.


<hr><div align='center'><b><a href='http://oeis.org/A120588'>A120588</a></b>: G.f. satisfies: 3*A(x) = 2 + x + A(x)^2, with a(0) = 1.<br></div>

by _Paul D. Hanna_, Jun 16 2006, Jan 24 2008

_Keywords_: `nonn,easy`

_Data_:

$$
\begin{array}{c|ccccccccccccccc }
    n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
    \hline
    A120588(n) & 1 & 1 & 1 & 2 & 5 & 14 & 42 & 132 & 429 & 1430 & 4862 & 16796 & 58786 & 208012 & 742900
\end{array}
$$
        

_Comments_:
   - This is essentially a duplicate of entry <a href="http://oeis.org/A000108">A000108</a>, the Catalan numbers (a(n) = <a href="http://oeis.org/A000108">A000108</a>(n-1) for n>0).
   - In order for the g.f. of an integer sequence to satisfy a functional equation of the form: r*A(x) = c + b*x + A(x)^n, where n > 1, it is necessary that the sequence start with [1, d, m*n*(n-1)/2], where d divides m*n*(n-1)/2 (m>0) and that the coefficients are given by r = n + d^2/m, c = r-1 and b = d^3/m. The remaining terms may then be integer and still satisfy: a_n(k) = r*a(k), where a_n(k) is the k-th term of the n-th self-convolution of the sequence.

_Formulae_:
   - G.f.: A(x) = 1 + Series_Reversion(1+3*x - (1+x)^2).
   - Lagrange Inversion yields g.f.: A(x) = Sum_{n>=0} C(2*n,n)/(n+1)*(2+x)^(n+1)/3^(2*n+1).
   - G.f.: (3 - sqrt(1-4*x))/2. - Maksym Voznyy (voznyy(AT)mail.ru), Aug 11 2009
   - a(n) = Sum_{k=1..n-1} a(k) * a(n-k) if n>1. - _Michael Somos_, Jul 23 2011
   - G.f.: 2 - G(0), where G(k)= 2*x*(2*k+1) + k +1 - 2*x*(k+1)*(2*k+3)/G(k+1); (continued fraction). - _Sergei N. Gladkovskii_, Jul 14 2013
   - G.f.: 2 - G(0), where G(k)= 1 - x/G(k+1) ; (continued fraction). - _Sergei N. Gladkovskii_, Jul 19 2013
   - a(n) ~ 2^(2*n-2)/(sqrt(Pi)*n^(3/2)). - _Vaclav Kotesovec_, Aug 19 2013
   - Given g.f. A(x), <a href="http://oeis.org/A001850">A001850</a>(n-1) = coefficient of x^n in A(x)^n if n>0, the derivative of log(A(x)) is the g.f. for <a href="http://oeis.org/A026641">A026641</a>. - _Michael Somos_, May 18 2015
   - A(x) = (1 + 2*Sum_{n >= 1} Catalan(n)*x^n)/(1 + Sum_{n >= 1} Catalan(n)*x^n) = (1 + 3/2*Sum_{n >= 1} binomial(2*n,n)*x^n )/(1 + Sum_{n >= 1} binomial(2*n,n)*x^n). - _Peter Bala_, Sep 01 2016

_Cross references_:
   - Cf. <a href="http://oeis.org/A120589">A120589</a> (A(x)^2); <a href="http://oeis.org/A120590">A120590</a> - <a href="http://oeis.org/A120607">A120607</a>.
   - Cf. <a href="http://oeis.org/A001850">A001850</a>, <a href="http://oeis.org/A026641">A026641</a>.


<hr><div align='center'><b><a href='http://oeis.org/A211216'>A211216</a></b>: Expansion of (1-8*x+21*x^2-20*x^3+5*x^4)/(1-9*x+28*x^2-35*x^3+15*x^4-x^5).<br></div>

by _Bruno Berselli_, May 11 2012

_Keywords_: `nonn,easy`

_Data_:

$$
\begin{array}{c|ccccccccccccccc }
    n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
    \hline
    A211216(n) & 1 & 1 & 2 & 5 & 14 & 42 & 132 & 429 & 1430 & 4862 & 16795 & 58766 & 207783 & 740924 & 2660139
\end{array}
$$
        

_Comments_:
   - In the paper of Kitaev, Remmel and Tiefenbruck (see the Links section), Q_(132)^(k,0,0,0)(x,0) represents a generating function depending on k and x.
   - For successive values ​​of k we have:
   - k=1, the g.f. of <a href="http://oeis.org/A000012">A000012</a>: 1/(1-x);
   - k=2,      "      <a href="http://oeis.org/A011782">A011782</a>: (1-x)/(1-2*x);
   - k=3,      "      <a href="http://oeis.org/A001519">A001519</a>: (1-2*x)/(1-3*x+x^2);
   - k=4,      "      <a href="http://oeis.org/A124302">A124302</a>: (1-3*x+x^2)/(1-4*x+3*x^2);
   - k=5,      "      <a href="http://oeis.org/A080937">A080937</a>: (1-4*x+3*x^2)/(1-5*x+6*x^2-x^3);
   - k=6,      "      <a href="http://oeis.org/A024175">A024175</a>: (1-5*x+6*x^2-x^3)/(1-6*x+10*x^2-4*x^3);
   - k=7,      "      <a href="http://oeis.org/A080938">A080938</a>: (1-6*x+10*x^2-4*x^3)/(1-7*x+15*x^2-10*x^3+x^4);
   - k=8,      "      <a href="http://oeis.org/A033191">A033191</a>: (1-7*x+15*x^2-10*x^3+x^4)/(1-8*x+21*x^2

_Formulae_:
   - G.f.: (1-3*x+x^2)*(1-5*x+5*x^2)/(1-9*x+28*x^2-35*x^3+15*x^4-x^5).
   - G.f.: 1/(1-x/(1-x/(1-x/(1-x/(1-x/(1-x/(1-x/(1-x/(1-x))))))))). - _Philippe Deléham_, Mar 14 2013

_Cross references_:
   - Cf. <a href="http://oeis.org/A001519">A001519</a>, <a href="http://oeis.org/A011782">A011782</a>, <a href="http://oeis.org/A024175">A024175</a>, <a href="http://oeis.org/A033191">A033191</a>, <a href="http://oeis.org/A080937">A080937</a>, <a href="http://oeis.org/A080938">A080938</a>, <a href="http://oeis.org/A124302">A124302</a>.
   - Cf. square array in <a href="http://oeis.org/A080934">A080934</a>.


<hr><div align='center'><b><a href='http://oeis.org/A033191'>A033191</a></b>: Binomial transform of [ 1, 0, 1, 1, 3, 6, 15, 36, 91, 231, 595,... ], which is essentially binomial(fibonacci(k) + 1, 2).<br></div>

by Simon Norton (simon(AT)dpmms.cam.ac.uk)

_Keywords_: `nonn,easy`

_Data_:

$$
\begin{array}{c|ccccccccccccccc }
    n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
    \hline
    A033191(n) & 1 & 1 & 2 & 5 & 14 & 42 & 132 & 429 & 1430 & 4861 & 16778 & 58598 & 206516 & 732825 & 2613834
\end{array}
$$
        

_Comments_:
   - Number of (s(0), s(1), ..., s(2n)) such that 0 < s(i) < 10 and |s(i) - s(i-1)| = 1 for i = 1,2,....,2n, s(0) = 1, s(2n) = 1. - _Herbert Kociemba_, Jun 14 2004
   - The sequence 1,2,5,14,... has g.f. 1/(1-2x-x^2/(1-2x-x^2/(1-2x-x^2/(1-2x)))) = (1-6x+10x^2-4x^3)/(1-8x+21x^2-20x^3+5x^4), and is the second binomial transform <a href="http://oeis.org/A001519">A001519</a> aerated. - _Paul Barry_, Dec 17 2009
   - Counts all paths of length (2*n), n>=0, starting and ending at the initial node on the path graph P_9, see the Maple program. - _Johannes W. Meijer_, May 29 2010

_Formulae_:
   - G.f.: (1-7x+15x^2-10x^3+x^4)/(1-8x+21x^2-20x^3+5x^4). - _Ralf Stephan_, May 13 2003
   - a(n) = (1/5)*sum(r, 1, 9, sin(r*Pi/10)^2(2*cos(r*Pi/10))^(2n)), n>=1; a(n) = 8*a(n-1)-21*a(n-2)+20*a(n-3)-5*a(n-4), n>=5. - _Herbert Kociemba_, Jun 14 2004
   - G.f.: 1 / (1 - x / (1 - x / (1 - x / (1 - x / (1 - x / (1 - x / (1 - x / (1 - x )))))))). - _Michael Somos_, May 12 2012

_Cross references_:
   - Cf. <a href="http://oeis.org/A033192">A033192</a>.
   - Cf. <a href="http://oeis.org/A081567">A081567</a>, <a href="http://oeis.org/A147748">A147748</a> and <a href="http://oeis.org/A178381">A178381</a>.
   - Cf. <a href="http://oeis.org/A211216">A211216</a>.



### Search by open content

In [58]:
searchable = search(query="pascal", table=True)

●

In [59]:
searchable(comment=lambda i,c: i in range(10))

_Results for query: <a href='https://oeis.org/search?fmt=json&start=0&q=pascal+keyword%3Atabl'>https://oeis.org/search?fmt=json&start=0&q=pascal+keyword%3Atabl</a>_<br><hr><div align='center'><b><a href='http://oeis.org/A007318'>A007318</a></b>: Pascal's triangle read by rows: C(n,k) = binomial(n,k) = n!/(k!*(n-k)!), 0 <= k <= n.<br></div>

by _N. J. A. Sloane_ and _Mira Bernstein_, Apr 28 1994

_Keywords_: `nonn,tabl,nice,easy,core,look,hear`

_Data_:

$$
\begin{array}{c|ccccccccccc }
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & 1 & 1 &  &  &  &  &  &  &  & \\2 & 1 & 2 & 1 &  &  &  &  &  &  & \\3 & 1 & 3 & 3 & 1 &  &  &  &  &  & \\4 & 1 & 4 & 6 & 4 & 1 &  &  &  &  & \\5 & 1 & 5 & 10 & 10 & 5 & 1 &  &  &  & \\6 & 1 & 6 & 15 & 20 & 15 & 6 & 1 &  &  & \\7 & 1 & 7 & 21 & 35 & 35 & 21 & 7 & 1 &  & \\8 & 1 & 8 & 28 & 56 & 70 & 56 & 28 & 8 & 1 & \\9 & 1 & 9 & 36 & 84 & 126 & 126 & 84 & 36 & 9 & 1
\end{array}
$$
        

_Comments_:
   - C(n,k) = number of k-element subsets of an n-element set.
   - Row n gives coefficients in expansion of (1+x)^n.
   - Binomial(n+k-1,n-1) is the number of ways of placing k indistinguishable balls into n boxes (the "bars and stars" argument - see Feller).
   - Binomial(n-1,k-1) is the number of compositions (ordered partitions) of n with k summands.
   - Binomial(n+k-1,k-1) is the number of weak compositions (ordered weak partitions) of n into exactly k summands.- _Juergen Will_, Jan 23 2016
   - Binomial(n,k) is the number of lattice paths from (0,0) to (n,k) using steps (1,0) and (1,1). - _Joerg Arndt_, Jul 01 2011
   - If thought of as an infinite lower triangular matrix, inverse begins:
   -   +1
   -   -1 +1
   -   +1 -2 +1

_Formulae_:
   - a(n, k) = C(n,k) = binomial(n, k).
   - C(n, k) = C(n-1, k) + C(n-1, k-1).
   - a(n+1, m) = a(n, m) + a(n, m-1), a(n, -1) := 0, a(n, m) := 0, n<m; a(0, 0)=1.
   - C(n, k)=n!/(k!(n-k)!) if 0<=k<=n, otherwise 0.
   - G.f.: 1/(1-y-x*y) = Sum(C(n, k)*x^k*y^n, n, k>=0)
   - G.f.: 1/(1-x-y) = Sum(C(n+k, k)*x^k*y^n, n, k>=0).
   - G.f. for row n: (1+x)^n = Sum_{k=0..n} C(n, k)x^k.
   - G.f. for column n: x^n/(1-x)^n.
   - E.g.f.: A(x, y) = exp(x+x*y).
   - E.g.f. for column n: x^n*exp(x)/n!.
   - In general the m-th power of <a href="http://oeis.org/A007318">A007318</a> is given by: T(0, 0) = 1, T(n, k) = T(n-1, k-1) + m*T(n-1, k), where n is the row-index and k is the column; also T(n, k) = m^(n-k) C(n, k).
   - Triangle T(n, k) read by rows; given by <a href="http://oeis.org/A000007">A000007</a> DELTA <a href="http://oeis.org/A000007">A000007</a>, where DELTA is Deléham's operator defined in <a href="http://oeis.org/A084938">A084938</a>.
   - Let P(n+1) = the number of integer partitions of (n+1); let p(i) = the number of parts of the i-th partition of (n+1); let d(i) = the number of different parts of the i-th partition of (n+1); let m(i, j) = multiplicity of the j-th part of the i-th partition of (n+1). Define the operator Sum_{i=1..P(n+1), p(i)=k+1} as the sum running from i=1 to i=P(n+1) but taking only partitions with p(i)=(k+1) parts into account. Define the operator Product_{j=1..d(i)} = product running from j=1 to j=d(i). Then C(n, k) = Sum_{p(i)=(k+1), i=1..P(n+1)} p(i)! / [Product_{j=1..d(i)} m(i, j)!]. E.g., C(5, 3) = 10 because n=6 has the following partitions with m=3 parts: (114), (123), (222). For their multiplicities one has: (114): 3!/(2!*1!) = 3; (123): 3!/(1!*1!*1!) = 6; (222): 3!/3! = 1. The sum is 3 + 6 + 1 = 10 = C(5, 3). - _Thomas Wieder_, Jun 03 2005
   - C(n, k) = Sum_{j=0..k} = (-1)^j*C(n+1+j, k-j)*<a href="http://oeis.org/A000108">A000108</a>(j). - _Philippe Deléham_, Oct 10 2005
   - G.f.: 1 + x(1 + x) + x^3(1 + x)^2 + x^6(1 + x)^3 + ... . - _Michael Somos_, Sep 16 2006
   - Sum_{k=0..floor(n/2)} x^(n-k)*T(n-k,k) = <a href="http://oeis.org/A000007">A000007</a>(n), <a href="http://oeis.org/A000045">A000045</a>(n+1), <a href="http://oeis.org/A002605">A002605</a>(n), <a href="http://oeis.org/A030195">A030195</a>(n+1), <a href="http://oeis.org/A057087">A057087</a>(n), <a href="http://oeis.org/A057088">A057088</a>(n), <a href="http://oeis.org/A057089">A057089</a>(n), <a href="http://oeis.org/A057090">A057090</a>(n), <a href="http://oeis.org/A057091">A057091</a>(n), <a href="http://oeis.org/A057092">A057092</a>(n), <a href="http://oeis.org/A057093">A057093</a>(n) for x = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, respectively. Sum_{k=0..floor(n/2)} (-1)^k*x^(n-k)*T(n-k,k) = <a href="http://oeis.org/A000007">A000007</a>(n), <a href="http://oeis.org/A010892">A010892</a>(n), <a href="http://oeis.org/A009545">A009545</a>(n+1), <a href="http://oeis.org/A057083">A057083</a>(n), <a href="http://oeis.org/A001787">A001787</a>(n+1), <a href="http://oeis.org/A030191">A030191</a>(n), <a href="http://oeis.org/A030192">A030192</a>(n), <a href="http://oeis.org/A030240">A030240</a>(n), <a href="http://oeis.org/A057084">A057084</a>(n), <a href="http://oeis.org/A057085">A057085</a>(n+1), <a href="http://oeis.org/A057086">A057086</a>(n), <a href="http://oeis.org/A084329">A084329</a>(n+1) for x = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, respectively. - _Philippe Deléham_, Sep 16 2006
   - C(n,k) <= <a href="http://oeis.org/A062758">A062758</a>(n) for n > 1. - _Reinhard Zumkeller_, Mar 04 2008
   - C(t+p-1, t) = Sum_{i=0..t} C(i+p-2, i) = Sum_{i=1..p} C(i+t-2, t-1). A binomial number is the sum of its left parent and all its right ancestors, which equals the sum of its right parent and all its left ancestors. - Lee Naish (lee(AT)cs.mu.oz.au), Mar 07 2008
   - From _Paul D. Hanna_, Mar 24 2011: 
       - Let A(x) = Sum_{n>=0} x^(n(n+1)/2)*(1+x)^n be the g.f. of the flattened triangle:
       - A(x) = 1 + (x + x^2) + (x^3 + 2*x^4 + x^5) + (x^6 + 3*x^7 + 3*x^8 + x^9) +...
       - then A(x) equals the series Sum_{n>=0} (1+x)^n*x^n*Product_{k=1..n} (1-(1+x)*x^(2k-1))/(1-(1+x)*x^(2k));
       - also, A(x) equals the continued fraction 1/(1- x*(1+x)/(1+ x*(1-x)*(1+x)/(1- x^3*(1+x)/(1+ x^2*(1-x^2)*(1+x)/(1- x^5*(1+x)/(1+ x^3*(1-x^3)*(1+x)/(1- x^7*(1+x)/(1+ x^4*(1-x^4)*(1+x)/(1- ...))))))))).
       - These formulas are due to (1) a q-series identity and (2) a partial elliptic theta function expression. 
   - Row n of the triangle is the result of applying the ConvOffs transform to the first n terms of the natural numbers (1, 2, 3, ..., n). See <a href="http://oeis.org/A001263">A001263</a> or <a href="http://oeis.org/A214281">A214281</a> for a definition of this transformation. - _Gary W. Adamson_, Jul 12 2012
   - From _L. Edson Jeffery_, Aug 02 2012: 
       - Row n (n >= 0) of the triangle is given by the n-th antidiagonal of the infinite matrix P^n, where P = (p_{i,j}), i,j >= 0, is the production matrix
       - 0, 1,
       - 1, 0, 1,
       - 0, 1, 0, 1,
       - 0, 0, 1, 0, 1,
       - 0, 0, 0, 1, 0, 1,
       - 0, 0, 0, 0, 1, 0, 1,
       - 0, 0, 0, 0, 0, 1, 0, 1,
       - 0, 0, 0, 0, 0, 0, 1, 0, 1,
       - ... 
   - Row n of the triangle is also given by the n+1 coefficients of the polynomial P_n(x) defined by the recurrence P_0(x) = 1, P_1(x) = x + 1, P_n(x) = x*P_{n-1}(x) + P_{n-2}(x), n > 1. - _L. Edson Jeffery_, Aug 12 2013
   - For a closed-form formula for arbitrary left and right borders of Pascal-like triangles see <a href="http://oeis.org/A228196">A228196</a>. - _Boris Putievskiy_, Aug 18 2013
   - For a closed-form formula for generalized Pascal's triangle see <a href="http://oeis.org/A228576">A228576</a>. - _Boris Putievskiy_, Sep 04 2013
   - (1+x)^n = Sum_{k=0..n} (-1)^(n-k)*binomial(n,k)*Sum_{i=0..k} k^(n-i)*binomial(k,i)*x^(n-i)/(n-i)!. - _Vladimir Kruchinin_, Oct 21 2013
   - E.g.f.: A(x,y) = exp(x+x*y) = 1 + (x+y*x)/( E(0)-(x+y*x)), where E(k) = 1 + (x+y*x)/(1 + (k+1)/E(k+1) ); (continued fraction). - _Sergei N. Gladkovskii_, Nov 08 2013
   - E.g.f.:  E(0) -1, where E(k) = 2 + x*(1+y)/(2*k+1 - x*(1+y)/E(k+1) ); (continued fraction). - _Sergei N. Gladkovskii_, Dec 24 2013
   - G.f.: 1 + x*(1+x)*(1+x^2*(1+x)/(W(0)-x^2-x^3)), where W(k) = 1 + (1+x)*x^(k+2) - (1+x)*x^(k+3)/W(k+1); (continued fraction). - _Sergei N. Gladkovskii_, Dec 24 2013
   - Sum_{n>=0} C(n,k)/n! = e/k!, where e = exp(1), while allowing n < k where C(n,k) = 0. Also Sum_{n>=0} C(n+k-1,k)/n! = e * <a href="http://oeis.org/A000262">A000262</a>(k)/k!, and for k>=1 equals e * <a href="http://oeis.org/A067764">A067764</a>(k)/<a href="http://oeis.org/A067653">A067653</a>(k). - _Richard R. Forberg_, Jan 01 2014
   - Sum_{n>=k} 1/C(n,k) = k/(k-1) for k>=1. - _Richard R. Forberg_, Feb 10 2014
   - From _Tom Copeland_, Apr 17 and 26 2014: 
       - Multiply each n-th diagonal of the Pascal lower triangular matrix by x^n and designate the result by <a href="http://oeis.org/A007318">A007318</a>(x) = P(x). Then with :xD:^n = x^n*(d/dx)^n and B(n,x), the Bell polynomials (<a href="http://oeis.org/A008277">A008277</a>),
       - A) P(x)= exp(x*dP) = exp[x*(e^M-I)] = exp[M*B(.,x)] = (I+dP)^B(.,x)
       -    with dP = <a href="http://oeis.org/A132440">A132440</a>, M = <a href="http://oeis.org/A238385">A238385</a>-I, and I = identity matrix, and
       - B) P(:xD:) = exp(dP:xD:) = exp[(e^M-I):xD:] = exp[M*B(.,:xD:)] = exp[M*xD] = (I+dP)^(xD) with action P(:xD:)g(x) = exp(dP:xD:)g(x) = g[(I+dP)*x] (cf. also <a href="http://oeis.org/A238363">A238363</a>).
       - C) P(x)^y = P(y*x). P(2x) = <a href="http://oeis.org/A038207">A038207</a>(x) = exp[M*B(.,2x)], the face vectors of the n-dim hypercubes.
       - D) P(x) = [St2]*exp(x*M)*[St1] = [St2]*(I+dP)^x*[St1]
       - E) = [St1]^(-1)*(I+dP)^x*[St1] = [St2]*(I+dP)^x*[St2]^(-1)
       - where [St1]=padded <a href="http://oeis.org/A008275">A008275</a> just as [St2]=<a href="http://oeis.org/A048993">A048993</a>=padded <a href="http://oeis.org/A008277">A008277</a> and exp(x*M) = (I+dP)^x = sum(k=0,..,infinity, C(x,k) dP^k). 
   - From _Peter Bala_, Dec 21 2014: 
       - Recurrence equation: T(n,k) = T(n-1,k)*(n + k)/(n - k) - T(n-1,k-1) for n >= 2 and 1 <= k < n, with boundary conditions T(n,0) = T(n,n) = 1. Note, changing the minus sign in the recurrence to a plus sign gives a recurrence for the square of the binomial coefficients - see <a href="http://oeis.org/A008459">A008459</a>.
       - There is a relation between the e.g.f.'s of the rows and the diagonals of the triangle, namely, exp(x) * e.g.f. for row n = e.g.f. for diagonal n. For example, for n = 3 we have exp(x)*(1 + 3*x + 3*x^2/2! + x^3/3!) = 1 + 4*x + 10*x^2/2! + 20*x^3/3! + 35*x^4/4! + .... This property holds more generally for the Riordan arrays of the form ( f(x), x/(1 - x) ), where f(x) is an o.g.f. of the form 1 + f_1*x + f_2*x^2 + .... See, for example, <a href="http://oeis.org/A055248">A055248</a> and <a href="http://oeis.org/A106516">A106516</a>.
       - Let P denote the present triangle. For k = 0,1,2,... define P(k) to be the lower unit triangular block array
       - /I_k 0\
       - \ 0  P/ having the k X k identity matrix I_k as the upper left block; in particular, P(0) = P. The infinite product P(0)*P(1)*P(2)*..., which is clearly well-defined, is equal to the triangle of Stirling numbers of the second kind <a href="http://oeis.org/A008277">A008277</a>. The infinite product in the reverse order, that is, ...*P(2)*P(1)*P(0), is equal to the triangle of Stirling cycle numbers <a href="http://oeis.org/A130534">A130534</a>. 
   - C(a+b,c) = Sum_{k=0..a} C(a,k)*C(b,b-c+k). This is a generalization of equation 1 from section 4.2.5 of the Prudnikov et al. reference, for a=b=c=n: C(2n,n) = Sum_{k=0..n} C(n,k)^2. See Links section for animation of new formula. - _Hermann Stamm-Wilbrandt_, Aug 26 2015
   - The row polynomials of the Pascal matrix P(n,x) = (1+x)^n are related to the Bernoulli polynomials Br(n,x) and their umbral compositional inverses Bv(n,x) by the umbral relation P(n,x) = (-Br(.,-Bv(.,x)))^n = (-1)^n Br(n,-Bv(.,x)), which translates into the matrix relation P = M * Br * M * Bv, where P is the Pascal matrix, M is the diagonal matrix diag(1,-1,1,-1,...), Br is the matrix for the coefficients of the Bernoulli polynomials, and Bv that for the umbral inverse polynomials defined umbrally by Br(n,Bv(.,x)) = x^n = Bv(n,Br(.,x)). Note M = M^(-1). - _Tom Copeland_, Sep 05 2015
   - 1/(1-x)^k = (r(x) * r(x^2) * r(x^4) * ...) where r(x) = (1+x)^k. - _Gary W. Adamson_, Oct 17 2016

_Cross references_:
   - Equals differences between consecutive terms of <a href="http://oeis.org/A102363">A102363</a>. - David G. Williams (davidwilliams(AT)Paxway.com), Jan 23 2006
   - Row sums give <a href="http://oeis.org/A000079">A000079</a> (powers of 2).
   - Cf. <a href="http://oeis.org/A083093">A083093</a> (triangle read mod 3), <a href="http://oeis.org/A214292">A214292</a> (first differences of rows).
   - Partial sums of rows give triangle <a href="http://oeis.org/A008949">A008949</a>.
   - Infinite matrix squared: <a href="http://oeis.org/A038207">A038207</a>, cubed: <a href="http://oeis.org/A027465">A027465</a>.
   - Cf. <a href="http://oeis.org/A101164">A101164</a>. If rows are sorted we get <a href="http://oeis.org/A061554">A061554</a> or <a href="http://oeis.org/A107430">A107430</a>.
   - Another version: <a href="http://oeis.org/A108044">A108044</a>.
   - Cf. <a href="http://oeis.org/A008277">A008277</a>, <a href="http://oeis.org/A132311">A132311</a>, <a href="http://oeis.org/A132312">A132312</a>, <a href="http://oeis.org/A052216">A052216</a>, <a href="http://oeis.org/A052217">A052217</a>, <a href="http://oeis.org/A052218">A052218</a>, <a href="http://oeis.org/A052219">A052219</a>, <a href="http://oeis.org/A052220">A052220</a>, <a href="http://oeis.org/A052221">A052221</a>, <a href="http://oeis.org/A052222">A052222</a>, <a href="http://oeis.org/A052223">A052223</a>, <a href="http://oeis.org/A144225">A144225</a>, <a href="http://oeis.org/A202750">A202750</a>, <a href="http://oeis.org/A211226">A211226</a>, <a href="http://oeis.org/A047999">A047999</a>, <a href="http://oeis.org/A026729">A026729</a>, <a href="http://oeis.org/A052553">A052553</a>, <a href="http://oeis.org/A051920">A051920</a>, <a href="http://oeis.org/A193242">A193242</a>.
   - Triangle sums (see the comments): <a href="http://oeis.org/A000079">A000079</a> (Row1); <a href="http://oeis.org/A000007">A000007</a> (Row2); <a href="http://oeis.org/A000045">A000045</a> (Kn11 & Kn21); <a href="http://oeis.org/A000071">A000071</a> (Kn12 & Kn22); <a href="http://oeis.org/A001924">A001924</a> (Kn13 & Kn23); <a href="http://oeis.org/A014162">A014162</a> (Kn14 & Kn24); <a href="http://oeis.org/A014166">A014166</a> (Kn15 & Kn25); <a href="http://oeis.org/A053739">A053739</a> (Kn16 & Kn26); <a href="http://oeis.org/A053295">A053295</a> (Kn17 & Kn27); <a href="http://oeis.org/A053296">A053296</a> (Kn18 & Kn28); <a href="http://oeis.org/A053308">A053308</a> (Kn19 & Kn29); <a href="http://oeis.org/A053309">A053309</a> (Kn110 & Kn210); <a href="http://oeis.org/A001519">A001519</a> (Kn3 & Kn4); <a href="http://oeis.org/A011782">A011782</a> (Fi1 & Fi2); <a href="http://oeis.org/A000930">A000930</a> (Ca1 & Ca2); <a href="http://oeis.org/A052544">A052544</a> (Ca3 & Ca4); <a href="http://oeis.org/A003269">A003269</a> (Gi1 & Gi2); <a href="http://oeis.org/A055988">A055988</a> (Gi3 & Gi4); <a href="http://oeis.org/A034943">A034943</a> (Ze1 & Ze2); <a href="http://oeis.org/A005251">A005251</a> (Ze3 & Ze4). - _Johannes W. Meijer_, Sep 22 2010
   - Fibonacci-Pascal triangles: <a href="http://oeis.org/A027926">A027926</a>, <a href="http://oeis.org/A036355">A036355</a>, <a href="http://oeis.org/A037027">A037027</a>, <a href="http://oeis.org/A074829">A074829</a>, <a href="http://oeis.org/A105809">A105809</a>, <a href="http://oeis.org/A109906">A109906</a>, <a href="http://oeis.org/A111006">A111006</a>, <a href="http://oeis.org/A114197">A114197</a>, <a href="http://oeis.org/A162741">A162741</a>, <a href="http://oeis.org/A228074">A228074</a>, <a href="http://oeis.org/A228196">A228196</a>, <a href="http://oeis.org/A228576">A228576</a>.
   - Cf. <a href="http://oeis.org/A137948">A137948</a>, <a href="http://oeis.org/A245334">A245334</a>.
   - Cf. <a href="http://oeis.org/A085478">A085478</a>, <a href="http://oeis.org/A258993">A258993</a>.


<hr><div align='center'><b><a href='http://oeis.org/A047999'>A047999</a></b>: Sierpiński's [Sierpinski's] triangle (or gasket): triangle, read by rows, formed by reading Pascal's triangle mod 2.<br></div>

by _N. J. A. Sloane_

_Keywords_: `nonn,tabl,easy,nice`

_Data_:

$$
\begin{array}{c|ccccccccccc }
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & 1 & 1 &  &  &  &  &  &  &  & \\2 & 1 & 0 & 1 &  &  &  &  &  &  & \\3 & 1 & 1 & 1 & 1 &  &  &  &  &  & \\4 & 1 & 0 & 0 & 0 & 1 &  &  &  &  & \\5 & 1 & 1 & 0 & 0 & 1 & 1 &  &  &  & \\6 & 1 & 0 & 1 & 0 & 1 & 0 & 1 &  &  & \\7 & 1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 &  & \\8 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & \\9 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1
\end{array}
$$
        

_Comments_:
   - Restored the alternative spelling of Sierpinski to facilitate searching for this triangle using regular-expression matching commands in ASCII. - _N. J. A. Sloane_, Jan 18 2016
   - Also triangle giving successive states of cellular automaton generated by "Rule 60" and "Rule 102". - _Hans Havermann_, May 26 2002
   - Also triangle formed by reading triangle of Eulerian numbers (A08292) mod 2. - _Philippe Deléham_, Oct 02 2003
   - Self-inverse when regarded as an infinite lower triangular matrix over GF(2).
   - Start with [1], repeatedly apply the map 0 -> [00/00], 1 -> [10/11] [Allouche and Berthe]
   - Also triangle formed by reading triangles <a href="http://oeis.org/A011117">A011117</a>, <a href="http://oeis.org/A028338">A028338</a>, <a href="http://oeis.org/A039757">A039757</a>, <a href="http://oeis.org/A059438">A059438</a>, <a href="http://oeis.org/A085881">A085881</a>, <a href="http://oeis.org/A086646">A086646</a>, <a href="http://oeis.org/A086872">A086872</a>, <a href="http://oeis.org/A087903">A087903</a>, <a href="http://oeis.org/A010421">A010421</a>9 mod 2. - _Philippe Deléham_, Jun 18 2005
   - J. H. Conway writes (in Math Forum): at least the first 31 rows give odd-sided constructible polygons (sides 1, 3, 5, 15,17 ... see <a href="http://oeis.org/A001317">A001317</a>). The 1's form a Sierpiński sieve. - M. Dauchez (mdzzdm(AT)yahoo.fr), Sep 19 2005
   - When regarded as an infinite lower triangular matrix, its inverse is a (-1,0,1)-matrix with zeros undisturbed and the nonzero entries in every column form the Prouhet-Thue-Morse sequence (1,-1,-1,1,-1,1,1,-1,...) <a href="http://oeis.org/A010060">A010060</a> (up to relabeling). - _David Callan_, Oct 27 2006
   - Triangle read by rows: antidiagonals of an array formed by successive iterates of running sums mod 2, beginning with (1, 1, 1,...). - _Gary W. Adamson_, Jul 10 2008
   - T(n,k) = <a href="http://oeis.org/A057427">A057427</a>(<a href="http://oeis.org/A143333">A143333</a>(n,k)). - _Reinhard Zumkeller_, Oct 24 2010

_Formulae_:
   - Lucas's Theorem is that T(n,k) = 1 if and only if the 1's in the binary expansion of k are a subset of the 1's in the binary expansion of n; or equivalently, k AND NOT n is zero, where AND and NOT are bitwise operators. - _Chai Wah Wu_, Feb 09 2016 and _N. J. A. Sloane_, Feb 10 2016
   - Sum_{k>=0} T(n, k) = <a href="http://oeis.org/A001316">A001316</a>(n) = 2^<a href="http://oeis.org/A000120">A000120</a>(n).
   - T(n,k) = T(n-1,k-1) XOR T(n-1,k), 0<k<n; T(n,0) = T(n,n) = 1. - _Reinhard Zumkeller_, Dec 13 2009
   - From _Vladimir Shevelev_, Dec 31 2013: 
       - For polynomial {s_n(x)} we have
       - s_0(x)=1; for n>=1, s_n(x) = prod{i=1,...,<a href="http://oeis.org/A000120">A000120</a>(n)}(x^(2^k_i) + 1),
       - if the binary expansion of n is n = sum{i=1,...,<a href="http://oeis.org/A000120">A000120</a>(n)}2^k_i;
       - G.f. sum {n>=0} s_n(x)*z^n = prod{k>=0}(1 + (x^(2^k)+1)z^(2^k)) (0<z<1/x).
       - Let x>1, t>0 be real numbers. Then
       - sum{n>=0}1/s_n(x)^t = prod{k>=0}(1 + 1/(x^(2^k)+1)^t);
       - sum{n>=0}(-1)^<a href="http://oeis.org/A000120">A000120</a>(n)/s_n(x)^t = prod{k>=0}(1 - 1/(x^(2^k)+1)^t).
       - In particular, for t=1, x>1, we have
       - sum{n>=0}(-1)^<a href="http://oeis.org/A000120">A000120</a>(n)/s_n(x) = 1 - 1/x. 

_Cross references_:
   - Other versions: <a href="http://oeis.org/A090971">A090971</a>, <a href="http://oeis.org/A038183">A038183</a>.
   - Cf. <a href="http://oeis.org/A007318">A007318</a>, <a href="http://oeis.org/A054431">A054431</a>, <a href="http://oeis.org/A001317">A001317</a>, <a href="http://oeis.org/A008292">A008292</a>, <a href="http://oeis.org/A083093">A083093</a>, <a href="http://oeis.org/A034931">A034931</a>, <a href="http://oeis.org/A034930">A034930</a>, <a href="http://oeis.org/A008975">A008975</a>, <a href="http://oeis.org/A034932">A034932</a>, <a href="http://oeis.org/A166360">A166360</a>, <a href="http://oeis.org/A249133">A249133</a>, <a href="http://oeis.org/A064194">A064194</a>, <a href="http://oeis.org/A227133">A227133</a>.
   - From _Johannes W. Meijer_, Jun 05 2011: 
       - <a href="http://oeis.org/A106344">A106344</a> is a skew version of this triangle.
       - Triangle sums (see the comments): <a href="http://oeis.org/A001316">A001316</a> (Row1; Related to Row2), <a href="http://oeis.org/A002487">A002487</a> (Related to Kn11, Kn12, Kn13, Kn21, Kn22, Kn23), <a href="http://oeis.org/A007306">A007306</a> (Kn3, Kn4), <a href="http://oeis.org/A060632">A060632</a> (Fi1, Fi2), <a href="http://oeis.org/A120562">A120562</a> (Ca1, Ca2), <a href="http://oeis.org/A112970">A112970</a> (Gi1, Gi2), <a href="http://oeis.org/A127830">A127830</a> (Ze3, Ze4). 


<hr><div align='center'><b><a href='http://oeis.org/A029635'>A029635</a></b>: The (1,2)-Pascal triangle (or Lucas triangle) read by rows.<br></div>

by _Mohammad K. Azarian_

_Keywords_: `nonn,tabl,nice,easy`

_Data_:

$$
\begin{array}{c|ccccccccccc }
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 2 &  &  &  &  &  &  &  &  & \\1 & 1 & 2 &  &  &  &  &  &  &  & \\2 & 1 & 3 & 2 &  &  &  &  &  &  & \\3 & 1 & 4 & 5 & 2 &  &  &  &  &  & \\4 & 1 & 5 & 9 & 7 & 2 &  &  &  &  & \\5 & 1 & 6 & 14 & 16 & 9 & 2 &  &  &  & \\6 & 1 & 7 & 20 & 30 & 25 & 11 & 2 &  &  & \\7 & 1 & 8 & 27 & 50 & 55 & 36 & 13 & 2 &  & \\8 & 1 & 9 & 35 & 77 & 105 & 91 & 49 & 15 & 2 & \\9 & 1 & 10 & 44 & 112 & 182 & 196 & 140 & 64 & 17 & 2
\end{array}
$$
        

_Comments_:
   - Dropping the first term and changing the boundary conditions to T(n,1)=n, T(n,n-1)=2 (n>=2), T(n,n)=1 yields the number of nonterminal symbols (which generate strings of length k) in a certain context-free grammar in Chomsky normal form that generates all permutations of n symbols. Summation over k (1<=k<=n) results in <a href="http://oeis.org/A003945">A003945</a>. For the number of productions of this grammar: see <a href="http://oeis.org/A090327">A090327</a>. Example: 1; 2, 1; 3, 2, 1; 4, 5, 2, 1; 5, 9, 7, 2, 1; 6, 14, 16, 9, 2, 1; In addition to the example of <a href="http://oeis.org/A090327">A090327</a> we have T(3,3)=#{S}=1, T(3,2)=#{D,E}=2 and T(3,1)=#{A,B,C}=3. - _Peter R. J. Asveld_, Jan 29 2004
   - With T(0,0)=1 : Triangle T(n,k), read by rows, given by (1,0,0,0,0,0,0,0,) DELTA (2,-1,0,0,0,0,0,0,0,...) where DELTA is the operator defined in <a href="http://oeis.org/A084938">A084938</a>. - _Philippe Deléham_, Oct 10 2011
   - For n > 0: T(n,k) = <a href="http://oeis.org/A097207">A097207</a>(n-1,k), 0 <= k < n. - _Reinhard Zumkeller_, Mar 12 2012
   - Much as the original Pascal triangle gives the Fibonacci numbers as sums of its diagonals, this triangle gives the Lucas numbers (<a href="http://oeis.org/A000032">A000032</a>) as sums of its diagonals; see Posamentier & Lehmann (2007). - _Alonso del Arte_, Apr 09 2012
   - For n > 0: T(n,k) = <a href="http://oeis.org/A029600">A029600</a>(n,k) - <a href="http://oeis.org/A007318">A007318</a>(n,k), 0 <= k <= n. - _Reinhard Zumkeller_, Apr 16 2012
   - Riordan array ((2-x)/(1-x), x/(1-x)). - _Philippe Deléham_, Mar 15 2013
   - For a closed-form formula for generalized Pascal's triangle see <a href="http://oeis.org/A228576">A228576</a>.  - _Boris Putievskiy_, Sep 04 2013
   - It appears that for the infinite set of (1,N) Pascal's triangles, the binomial transform of the n-th row (n>0), followed by zeros, is equal to the n-th partial sum of (1, N, N, N, ...). Example: for the (1,2) Pascal's triangle, the binomial transform of the second row followed by zeros, i.e., of (1, 3, 2, 0, 0, 0, ...), is equal to the second partial sum of (1, 2, 2, 2, ...) = (1, 4, 9, 16, ...). - _Gary W. Adamson_, Aug 11 2015
   - Given any (1,N) Pascal triangle, let the binomial transform of the n-th row (n>1) followed by zeros be Q(x). It appears that the binomial transform of the (n-1)-th row prefaced by a zero is Q(n-1). Example: In the (1,2) Pascal triangle the binomial transform of row 3: (1, 4, 5, 2, 0, 0, 0, ...) is <a href="http://oeis.org/A000330">A000330</a> starting with 1: (1, 5, 14, 30, 55, 91, ...). The binomial transform of row 2 prefaced by a zero and followed by zeros, i.e., of (0, 1, 3, 2, 0, 0, 0, ...) is (0, 1, 5, 14, 30, 55, ...). - _Gary W. Adamson_, Sep 28 2015
   - It appears that in the array accompanying each (1,N) Pascal triangle (diagonals of the triangle), the binomial transform of (..., 1, N, 0, 0, 0, ...) preceded by (n-1) zeros generates the n-th row of the array (n>0). Then delete the zeros in the result. Example: in the (1,2) Pascal triangle, row 3 (1, 5, 14, 30, ...) is the binomial transform of (0, 0, 1, 2, 0, 0, 0, ...) with the resulting zeros deleted. - _Gary W. Adamson_, Oct 11 2015

_Formulae_:
   - T(n,k) = T(n-1, k-1) + T(n-1, k) = C(n, k) + C(n-1, k-1) = C(n, k)*(n+k)/n = <a href="http://oeis.org/A007318">A007318</a>(n, k) + <a href="http://oeis.org/A007318">A007318</a>(n-1, k-1) = <a href="http://oeis.org/A061896">A061896</a>(n+k, k) but with T(0, 0)=1 and T(1, 1)=2. Row sum is floor[3^2(n-1)] i.e., <a href="http://oeis.org/A003945">A003945</a>. - _Henry Bottomley_, Apr 26 2002
   - G.f.: 1 + (1 + x*y) / (1 - x - x*y). - _Michael Somos_, Jul 15 2003
   - G.f. for n-th row: (x+2*y)*(x+y)^(n-1).
   - O.g.f. for row n: (1+x)/(1-x)^(n+1). The entries in row n are the nonzero entries in column n of <a href="http://oeis.org/A053120">A053120</a> divided by 2^(n-1). - _Peter Bala_, Aug 14 2008
   - T(2n,n) - T(2n,n+1)= Catalan(n)= <a href="http://oeis.org/A000108">A000108</a>(n). - _Philippe Deléham_, Mar 19 2009
   - With T(0,0) = 1, as in the Example section below, this is known as Vieta's array. The LU factorization of the square array is given by Yang and Leida, equation 20. - _Peter Bala_, Feb 11 2012
   - exp(x) * e.g.f. for row n = e.g.f. for diagonal n. For example, for n = 3 we have exp(x)*(1 + 4*x + 5*x^2/2! + 2*x^3/3!) = 1 + 5*x + 14*x^2/2! + 30*x^3/3! + 55*x^4/4! + .... The same property holds more generally for Riordan arrays of the form ( f(x), x/(1 - x) ). - _Peter Bala_, Dec 22 2014
   - For n>=1: T(n,0) + T(n,1) + T(n,2) = <a href="http://oeis.org/A000217">A000217</a>(n+1). T(n,n-2) = (n-1)^2. - _Bob Selcoe_, Mar 29 2016:

_Cross references_:
   - Cf. <a href="http://oeis.org/A007318">A007318</a>, <a href="http://oeis.org/A034807">A034807</a>, <a href="http://oeis.org/A061896">A061896</a>, <a href="http://oeis.org/A029653">A029653</a> (row-reversed), <a href="http://oeis.org/A157000">A157000</a>.
   - Sums along ascending diagonals give Lucas numbers, n>0.
   - Cf. <a href="http://oeis.org/A090327">A090327</a>, <a href="http://oeis.org/A003945">A003945</a>, <a href="http://oeis.org/A029638">A029638</a>, <a href="http://oeis.org/A228196">A228196</a>, <a href="http://oeis.org/A228576">A228576</a>.
   - Cf. <a href="http://oeis.org/A000330">A000330</a>.
   - Cf. <a href="http://oeis.org/A000217">A000217</a>.


<hr><div align='center'><b><a href='http://oeis.org/A029653'>A029653</a></b>: Numbers in (2,1)-Pascal triangle (by row).<br></div>

by _Mohammad K. Azarian_

_Keywords_: `nonn,tabl`

_Data_:

$$
\begin{array}{c|ccccccccccc }
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & 2 & 1 &  &  &  &  &  &  &  & \\2 & 2 & 3 & 1 &  &  &  &  &  &  & \\3 & 2 & 5 & 4 & 1 &  &  &  &  &  & \\4 & 2 & 7 & 9 & 5 & 1 &  &  &  &  & \\5 & 2 & 9 & 16 & 14 & 6 & 1 &  &  &  & \\6 & 2 & 11 & 25 & 30 & 20 & 7 & 1 &  &  & \\7 & 2 & 13 & 36 & 55 & 50 & 27 & 8 & 1 &  & \\8 & 2 & 15 & 49 & 91 & 105 & 77 & 35 & 9 & 1 & \\9 & 2 & 17 & 64 & 140 & 196 & 182 & 112 & 44 & 10 & 1
\end{array}
$$
        

_Comments_:
   - Reverse of <a href="http://oeis.org/A029635">A029635</a>. Row sums are <a href="http://oeis.org/A003945">A003945</a>. Diagonal sums are Fib(n+2) = sum_{k=0..floor(n/2)} (2n-3k)C(n-k,n-2k)/(n-k). - _Paul Barry_, Jan 30 2005
   - Riordan array ((1+x)/(1-x), x/(1-x)). The signed triangle (-1)^(n-k)T(n,k) or ((1-x)/(1+x), x/(1+x)) is the inverse of <a href="http://oeis.org/A055248">A055248</a>. Row sums are <a href="http://oeis.org/A003945">A003945</a>. Diagonal sums are F(n+2). - _Paul Barry_, Feb 03 2005
   - Row sums = <a href="http://oeis.org/A003945">A003945</a>: (1, 3, 6, 12, 24, 48, 96...) = (1, 3, 7, 15, 31, 63, 127...) - (0, 0, 1, 3, 7, 15, 31,...); where (1, 3, 7, 15,...) = <a href="http://oeis.org/A000225">A000225</a>. - _Gary W. Adamson_, Apr 22 2007
   - Triangle T(n,k), read by rows, given by (2,-1,0,0,0,0,0,0,0,...) DELTA (1,0,0,0,0,0,0,0,0,...) where DELTA is the operator defined in <a href="http://oeis.org/A084938">A084938</a>. - _Philippe Deléham_, Nov 17 2011
   - <a href="http://oeis.org/A029653">A029653</a> is jointly generated with <a href="http://oeis.org/A208510">A208510</a> as an array of coefficients of polynomials v(n,x): initially, u(1,x)=v(1,x)=1; for n>1, u(n,x)=u(n-1,x)+x*v(n-1)x and v(n,x)=u(n-1,x)+x*v(n-1,x)+1. See the Mathematica section. - _Clark Kimberling_, Feb 28 2012
   - For a closed-form formula for arbitrary left and right borders of Pascal like triangle, see <a href="http://oeis.org/A228196">A228196</a>. - _Boris Putievskiy_, Aug 18 2013
   - For a closed-form formula for generalized Pascal's triangle, see <a href="http://oeis.org/A228576">A228576</a>. - _Boris Putievskiy_, Sep 04 2013

_Formulae_:
   - T(n, k) = C(n-2, k-1) + C(n-2, k) + C(n-1, k-1) + C(n-1, k).
   - G.f.: (1 + x + y + xy)/(1 - y - xy). - _Ralf Stephan_, May 17 2004
   - T(n, k) = (2n-k)*binomial(n, n-k)/n, n, k>0. - _Paul Barry_, Jan 30 2005
   - Sum_{k=0..n} T(n, k)*x^k gives <a href="http://oeis.org/A003945">A003945</a>-<a href="http://oeis.org/A003954">A003954</a> for x = 1, 2, 3, 4, 5, 6, 7, 8, 9, 10. - _Philippe Deléham_, Jul 10 2005
   - T(n, k) = C(n-1, k) + C(n, k) . - _Philippe Deléham_, Jul 10 2005
   - Equals <a href="http://oeis.org/A097806">A097806</a> * <a href="http://oeis.org/A007318">A007318</a>, i.e., the pairwise operator * Pascal's Triangle as infinite lower triangular matrices. - _Gary W. Adamson_, Apr 22 2007
   - From _Peter Bala_, Dec 27 2014: 
       - exp(x) * e.g.f. for row n = e.g.f. for diagonal n. For example, for n = 3 we have exp(x)*(2 + 5*x + 4*x^2/2! + x^3/3!) = 2 + 7*x + 16*x^2/2! + 30*x^3/3! + 50*x^4/4! + .... The same property holds more generally for Riordan arrays of the form ( f(x), x/(1 - x) ).
       - Let M denote the lower unit triangular array with 1's on the main diagonal and 1's everywhere else below the main diagonal except for the first column which consists of the sequence [1,2,2,2,....]. For k = 0,1,2,... define M(k) to be the lower unit triangular block array
       - /I_k 0\
       - \ 0  M/ having the k X k identity matrix I_k as the upper left block; in particular, M(0) = M. Then the present triangle equals the infinite product M(0)*M(1)*M(2)*... (which is clearly well-defined). See the Example section. 

_Cross references_:
   - (d, 1) Pascal triangles for d=3..10: <a href="http://oeis.org/A093560">A093560</a>-5, <a href="http://oeis.org/A093644">A093644</a>-5.
   - Cf. <a href="http://oeis.org/A007318">A007318</a>, <a href="http://oeis.org/A003945">A003945</a>, <a href="http://oeis.org/A208510">A208510</a>, <a href="http://oeis.org/A228196">A228196</a>, <a href="http://oeis.org/A228576">A228576</a>.
   - Cf. <a href="http://oeis.org/A078812">A078812</a>, <a href="http://oeis.org/A106195">A106195</a>.


<hr><div align='center'><b><a href='http://oeis.org/A074909'>A074909</a></b>: Running sum of Pascal's triangle (A007318), or beheaded Pascal's triangle read by beheaded rows.<br></div>

by _Wouter Meeussen_, Oct 01 2002

_Keywords_: `easy,nonn,tabl`

_Data_:

$$
\begin{array}{c|ccccccccccc }
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & 1 & 2 &  &  &  &  &  &  &  & \\2 & 1 & 3 & 3 &  &  &  &  &  &  & \\3 & 1 & 4 & 6 & 4 &  &  &  &  &  & \\4 & 1 & 5 & 10 & 10 & 5 &  &  &  &  & \\5 & 1 & 6 & 15 & 20 & 15 & 6 &  &  &  & \\6 & 1 & 7 & 21 & 35 & 35 & 21 & 7 &  &  & \\7 & 1 & 8 & 28 & 56 & 70 & 56 & 28 & 8 &  & \\8 & 1 & 9 & 36 & 84 & 126 & 126 & 84 & 36 & 9 & \\9 & 1 & 10 & 45 & 120 & 210 & 252 & 210 & 120 & 45 & 10
\end{array}
$$
        

_Comments_:
   - This sequence counts the "almost triangular" partitions of n. A partition is triangular if it is of the form 0+1+2+...+k. Examples: 3=0+1+2, 6=0+1+2+3. An "almost triangular" partition is a triangular partition with at most 1 added to each of the parts. Examples: 7 = 1+1+2+3 = 0+2+2+3 = 0+1+3+3 = 0+1+2+4. Thus a(7)=4. 8 = 1+2+2+3 = 1+1+3+3 = 1+1+2+4 = 0+2+3+3 = 0+2+2+4 = 0+1+3+4 so a(8)=6. - _Moshe Shmuel Newman_, Dec 19 2002
   - The "almost triangular" partitions are the ones cycled by the operation of "Bulgarian solitaire", as defined by Martin Gardner.
   - Start with <a href="http://oeis.org/A007318">A007318</a> - I (I = Identity matrix), then delete right border of zeros. - _Gary W. Adamson_, Jun 15 2007
   - Also the number of increasing acyclic functions from {1..n-k+1} to {1..n+2}. A function f is acyclic if for every subset B of the domain the image of B under f does not equal B. For example, T(3,1)=4 since there are exactly 4 increasing acyclic functions from {1,2,3} to {1,2,3,4,5}: f1={(1,2),(2,3),(3,4)}, f2={(1,2),(2,3),(3,5)}, f3={(1,2),(2,4),(3,5)} and f4={(1,3),(2,4),(4,5)}. - _Dennis P. Walsh_, Mar 14 2008
   - Second Bernoulli polynomials are (from <a href="http://oeis.org/A164555">A164555</a> instead of <a href="http://oeis.org/A027641">A027641</a>) B2(n,x) = 1; 1/2, 1; 1/6, 1, 1; 0, 1/2, 3/2, 1; -1/30, 0, 1, 2, 1; 0, -1/6, 0, 5/3, 5/2, 1; ... . Then (B2(n,x)/<a href="http://oeis.org/A002260">A002260</a>) = 1; 1/2, 1/2; 1/6, 1/2, 1/3; 0, 1/4, 1/2, 1/4; -1/30, 0, 1/3, 1/2, 1/5; 0, -1/12, 0, 5/12, 1/2, 1/6; ... . See (from Faulhaber 1631) Jacob Bernoulli Summae Potestatum (sum of powers) in <a href="http://oeis.org/A159688">A159688</a>. Inverse polynomials are 1; -1, 2; 1, -3, 3; -1, 4, -6, 4; ... = <a href="http://oeis.org/A074909">A074909</a> with negative even diagonals. Reflected <a href="http://oeis.org/A053382">A053382</a>/<a href="http://oeis.org/A053383">A053383</a> = reflected B(n,x) = RB(n,x) = 1; -1/2, 1; 1/6, -1, 1; 0, 1/2, -3/2, 1; ... . <a href="http://oeis.org/A074909">A074909</a> is inverse of RB(n,x)/<a href="http://oeis.org/A002260">A002260</a> = 1; -1/2, 1/2; 1/6, -1/2, 1/3; 0, 1/4, -1/2, 1/4; ... . - _Paul Curtz_, Jun 21 2010
   - <a href="http://oeis.org/A054143">A054143</a> is the fission of the polynomial sequence (p(n,x)) given by p(n,x) = x^n + x^(n-1) + ... + x + 1 by the polynomial sequence ((x+1)^n). See <a href="http://oeis.org/A193842">A193842</a> for the definition of fission. - _Clark Kimberling_, Aug 07 2011
   - Reversal of <a href="http://oeis.org/A135278">A135278</a>. - _Philippe Deléham_, Feb 11 2012
   - For a closed-form formula for arbitrary left and right borders of Pascal-like triangles see <a href="http://oeis.org/A228196">A228196</a>. - _Boris Putievskiy_, Aug 19 2013
   - For a closed-form formula for generalized Pascal's triangle see <a href="http://oeis.org/A228576">A228576</a>. - _Boris Putievskiy_, Sep 09 2013
   - From <a href="http://oeis.org/A238363">A238363</a>, the operator equation d/d(:xD:)f(xD)={exp[d/d(xD)]-1}f(xD) = f(xD+1)-f(xD) follows. Choosing f(x) = x^n and using :xD:^n/n! = Binom(xD,n) and (xD)^n = Bell(n,:xD:), the Bell polynomials of <a href="http://oeis.org/A008277">A008277</a>, it follows that the lower triangular matrix [padded <a href="http://oeis.org/A074909">A074909</a>]

_Formulae_:
   - T(n, k) = Sum_{i=0..n} C(i, n-k) = C(n+1, k).
   - Row n has g.f. (1+x)^(n+1)-x^(n+1).
   - T(n, k) = T(n-1, k-1) + T(n-1, k) for k: 0<k<n, T(n, 0)=1, T(n, n)=n. - _Reinhard Zumkeller_, Apr 18 2005
   - T(n,k) = T(n-1,k) + 2*T(n-1,k-1) - T(n-2,k-1) - T(n-2,k-2), T(0,0)=1, T(1,0)=1, T(1,1)=2, T(n,k)=0 if k<0 or if k>n. - _Philippe Deléham_, Dec 27 2013
   - G.f. for column k (with leading zeros): x^(k-1)*(1/(1-x)^(k+1)-1), k >= 0. - _Wolfdieter Lang_, Nov 04 2014
   - Up(n, x+y) = (Up(.,x)+ y)^n = Sum_{k=0..n} binomial(n,k) Up(k,x)*y^(n-k), where Up(n,x) = ((x+1)^(n+1)-x^(n+1)) / (n+1) = P(n,x)/(n+1) with P(n,x) the n-th row polynomial of this entry. dUp(n,x)/dx = n * Up(n-1,x) and dP(n,x)/dx = (n+1)*P(n-1,x). - _Tom Copeland_, Nov 14 2014
   - The o.g.f. GF(x,t) = x / ((1-t*x)*(1-(1+t)x)) = x + (1+2t)*x^2 + (1+3t+3t^2)*x^3 + ... has the inverse GFinv(x,t) = (1+(1+2t)x-sqrt(1+(1+2t)*2x+x^2))/(2t(1+t)x) in x about 0, which generates the row polynomials (mod row signs) of <a href="http://oeis.org/A033282">A033282</a>. The reciprocal of the o.g.f., i.e., x/GF(x,t), gives the free cumulants (1, -(1+2t) , t(1+t) , 0, 0, ...) associated with the moments defined by GFinv, and, in fact, these free cumulants generate these moments through the noncrossing partitions of <a href="http://oeis.org/A134264">A134264</a>. The associated e.g.f. and relations to Grassmannians are described in <a href="http://oeis.org/A248727">A248727</a>, whose polynomials are the basis for an Appell sequence of polynomials that are umbral compositional inverses of the Appell sequence formed from this entry's polynomials (distinct from the one described in the comments above, without the normalizing reciprocal). - _Tom Copeland_, Jan 07 2015

_Cross references_:
   - Cf. <a href="http://oeis.org/A007318">A007318</a>, <a href="http://oeis.org/A181971">A181971</a>, <a href="http://oeis.org/A228196">A228196</a>, <a href="http://oeis.org/A228576">A228576</a>.
   - Row sums are <a href="http://oeis.org/A000225">A000225</a>, diagonal sums are <a href="http://oeis.org/A052952">A052952</a>.
   - The number of acyclic functions is <a href="http://oeis.org/A058127">A058127</a>.
   - Cf. <a href="http://oeis.org/A008292">A008292</a>, <a href="http://oeis.org/A090582">A090582</a>, <a href="http://oeis.org/A019538">A019538</a>, <a href="http://oeis.org/A049019">A049019</a>, <a href="http://oeis.org/A133314">A133314</a>, <a href="http://oeis.org/A135278">A135278</a>, <a href="http://oeis.org/A133437">A133437</a>, <a href="http://oeis.org/A111785">A111785</a>, <a href="http://oeis.org/A126216">A126216</a>, <a href="http://oeis.org/A134685">A134685</a>, <a href="http://oeis.org/A133932">A133932</a>, <a href="http://oeis.org/A248727">A248727</a>, <a href="http://oeis.org/A033282">A033282</a>, <a href="http://oeis.org/A134264">A134264</a>.
   - Cf. <a href="http://oeis.org/A000027">A000027</a>, <a href="http://oeis.org/A000217">A000217</a>, <a href="http://oeis.org/A000292">A000292</a>, <a href="http://oeis.org/A000332">A000332</a>, <a href="http://oeis.org/A000389">A000389</a>, <a href="http://oeis.org/A000579">A000579</a>, <a href="http://oeis.org/A000580">A000580</a>, <a href="http://oeis.org/A000581">A000581</a>, <a href="http://oeis.org/A000582">A000582</a>.


<hr><div align='center'><b><a href='http://oeis.org/A008459'>A008459</a></b>: Square the entries of Pascal's triangle.<br></div>

by _N. J. A. Sloane_

_Keywords_: `nonn,tabl,easy`

_Data_:

$$
\begin{array}{c|ccccccccccc }
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & 1 & 1 &  &  &  &  &  &  &  & \\2 & 1 & 4 & 1 &  &  &  &  &  &  & \\3 & 1 & 9 & 9 & 1 &  &  &  &  &  & \\4 & 1 & 16 & 36 & 16 & 1 &  &  &  &  & \\5 & 1 & 25 & 100 & 100 & 25 & 1 &  &  &  & \\6 & 1 & 36 & 225 & 400 & 225 & 36 & 1 &  &  & \\7 & 1 & 49 & 441 & 1225 & 1225 & 441 & 49 & 1 &  & \\8 & 1 & 64 & 784 & 3136 & 4900 & 3136 & 784 & 64 & 1 & \\9 & 1 & 81 & 1296 & 7056 & 15876 & 15876 & 7056 & 1296 & 81 & 1
\end{array}
$$
        

_Comments_:
   - Number of lattice paths from (0, 0) to (n, n) with steps (1, 0) and (0, 1), having k right turns. - _Emeric Deutsch_, Nov 23 2003
   - Product of <a href="http://oeis.org/A007318">A007318</a> and <a href="http://oeis.org/A105868">A105868</a>. - _Paul Barry_, Nov 15 2005
   - Number of partitions that fit in an n X n box with Durfee square k. - _Franklin T. Adams-Watters_, Feb 20 2006
   - From _Peter Bala_, Oct 23 2008: 
       - Narayana numbers of type B. Row n of this triangle is the h-vector of the simplicial complex dual to an associahedron of type B_n (a cyclohedron) [Fomin & Reading, p. 60]. See <a href="http://oeis.org/A063007">A063007</a> for the corresponding f-vectors for associahedra of type B_n. See <a href="http://oeis.org/A001263">A001263</a> for the h-vectors for associahedra of type A_n. The Hilbert transform of this triangular array is <a href="http://oeis.org/A108625">A108625</a> (see <a href="http://oeis.org/A145905">A145905</a> for the definition of this term).
       - Let A_n be the root lattice generated as a monoid by {e_i - e_j: 0 <= i, j <= n + 1}. Let P(A_n) be the polytope formed by the convex hull of this generating set. Then the rows of this array are the h-vectors of a unimodular triangulation of P(A_n) [Ardila et al.]. <a href="http://oeis.org/A063007">A063007</a> is the corresponding array of f-vectors for these type A_n polytopes. See <a href="http://oeis.org/A086645">A086645</a> for the array of h-vectors for type C_n polytopes and <a href="http://oeis.org/A108558">A108558</a> for the array of h-vectors associated with type D_n polytopes.
   - The n-th row consists of the coefficients of the polynomial P_n(t) = int((1 + t^2 - 2*t*cos(s))^n, s = 0..2*Pi)/Pi/2. For example, when n = 3, we get P_3(t) = t^6 + 9*t^4 + 9*t^2 + 1; the coefficients are 1, 9, 9, 1. - _Theodore Kolokolnikov_, Oct 26 2010
   - Let E(y) = sum {n >= 0} y^n/n!^2 = BesselJ(0, 2*sqrt(-y)). Then this triangle is the generalized Riordan array (E(y), y) with respect to the sequence n!^2 as defined in Wang and Wang. - _Peter Bala_, Jul 24 2013

_Formulae_:
   - E.g.f.: exp((1+y)*x)*BesselI(0, 2*sqrt(y)*x). - _Vladeta Jovovic_, Nov 17 2003
   - G.f.: 1/sqrt(1-2*y-2*x*y+y^2-2*x*y^2+x^2*y^2); g.f. for row n: (1-t)^n P_n[(1+t)/(1-t)] where the P_n's are the Legendre polynomials. - _Emeric Deutsch_, Nov 23 2003
   - G.f. for column k is sum(C(k, j)^2*x^(k+j), j, 0, k)/(1-x)^(2k+1). - _Paul Barry_, Nov 15 2005
   - Column k has g.f. x^k*Legendre_P(k, (1+x)/(1-x))/(1-x)^(k+1) = x^k*sum{j = 0..k, C(k, j)^2*x^j}/(1-x)^(2k+1). - _Paul Barry_, Nov 19 2005
   - Let E be the operator D*x*D, where D denotes the derivative operator d/dx. Then 1/n!^2 * E^n(1/(1-x)) = (row n generating polynomial)/(1-x)^(2n+1) = sum_{k = 0..inf} binomial(n + k, k)^2*x^k. For example, when n = 3 we have 1/3!^2*E^3(1/(1-x)) = (1 + 9*x + 9*x^2 + x^3)/(1-x)^7 = 1/3!^2 * sum_{k = 0..inf} [(k+1)*(k+2)*(k+3)]^2*x^k. - _Peter Bala_, Oct 23 2008
   - G.f.: A(x, y) = Sum_{n >= 0} (2n)!/n!^2 * x^(2n)*y^n/(1-x-x*y)^(2n+1). - _Paul D. Hanna_, Oct 31 2010
   - From _Peter Bala_, Jul 24 2013: 
       - Let E(y) = sum_{n >= 0} y^n/n!^2 = BesselJ(0, 2*sqrt(-y)). Generating function: E(y)*E(x*y) = 1 + (1 + x)*y + (1 + 4*x + x^2)*y^2/2!^2 + (1 + 9*x + 9*x^2 + x^3)*y^3/3!^2 + .... Cf. the unsigned version of <a href="http://oeis.org/A021009">A021009</a> with generating function exp(y)*E(x*y).
       - The n-th power of this array has the generating function E(y)^n*E(x*y). In particular, the matrix inverse <a href="http://oeis.org/A055133">A055133</a> has the generating function E(x*y)/E(y). 
   - T(n,k) = T(n-1,k)*(n+k)/(n-k)+T(n-1,k-1), T(n,0)=T(n,n)=1. - _Vladimir Kruchinin_, Oct 18 2014
   - Observe that the recurrence T(n,k) = T(n-1,k)*(n+k)/(n-k) - T(n-1,k-1), for n >= 2 and 1 <= k < n, with boundary conditions T(n,0) = T(n,n) = 1 gives Pascal's triangle <a href="http://oeis.org/A007318">A007318</a>. - _Peter Bala_, Dec 21 2014
   - n-th row polynomial R(n,x) = [z^n] (1 + (1 + x)*z + x*z^2)^n. Note that 1/n*[z^(n-1)] (1 + (1 + x)*z + x*z^2)^n gives the row polynomials of <a href="http://oeis.org/A001263">A001263</a>. - _Peter Bala_, Jun 24 2015
   - Binomial transform of <a href="http://oeis.org/A105868">A105868</a>. If G(x,t) = 1/sqrt(1 - 2*(1 + t)*x + (1 - t)^2*x^2) denotes the o.g.f. of this array then 1 + x*d/dx(log(G(x,t)) = 1 + (1 + t)*x + (1 + 6*t + t^2)*x^2 + ... is the o.g.f. for <a href="http://oeis.org/A086645">A086645</a>. - _Peter Bala_, Sep 06 2015

_Cross references_:
   - Row sums are in <a href="http://oeis.org/A000984">A000984</a>. Columns 0-3 are <a href="http://oeis.org/A000012">A000012</a>, <a href="http://oeis.org/A000290">A000290</a>, <a href="http://oeis.org/A000537">A000537</a>, <a href="http://oeis.org/A001249">A001249</a>.
   - Cf. <a href="http://oeis.org/A007318">A007318</a>, <a href="http://oeis.org/A055133">A055133</a>, <a href="http://oeis.org/A116647">A116647</a>, <a href="http://oeis.org/A001263">A001263</a>, <a href="http://oeis.org/A086645">A086645</a>, <a href="http://oeis.org/A063007">A063007</a>, <a href="http://oeis.org/A108558">A108558</a>, <a href="http://oeis.org/A108625">A108625</a>(Hilbert transform), <a href="http://oeis.org/A145903">A145903</a>, <a href="http://oeis.org/A181543">A181543</a>, <a href="http://oeis.org/A086645">A086645</a> (logarithmic derivative), <a href="http://oeis.org/A105868">A105868</a> (inverse binomial transform).


<hr><div align='center'><b><a href='http://oeis.org/A037027'>A037027</a></b>: Skew Fibonacci-Pascal triangle read by rows.<br></div>

by _Floor van Lamoen_, Jan 01 1999

_Keywords_: `easy,nonn,tabl`

_Data_:

$$
\begin{array}{c|ccccccccccc }
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & 1 & 1 &  &  &  &  &  &  &  & \\2 & 2 & 2 & 1 &  &  &  &  &  &  & \\3 & 3 & 5 & 3 & 1 &  &  &  &  &  & \\4 & 5 & 10 & 9 & 4 & 1 &  &  &  &  & \\5 & 8 & 20 & 22 & 14 & 5 & 1 &  &  &  & \\6 & 13 & 38 & 51 & 40 & 20 & 6 & 1 &  &  & \\7 & 21 & 71 & 111 & 105 & 65 & 27 & 7 & 1 &  & \\8 & 34 & 130 & 233 & 256 & 190 & 98 & 35 & 8 & 1 & \\9 & 55 & 235 & 474 & 594 & 511 & 315 & 140 & 44 & 9 & 1
\end{array}
$$
        

_Comments_:
   - T(n,k) is the number of lattice paths from (0,0) to (n,k) using steps (0,1), (1,0), (2,0). - _Joerg Arndt_, Jun 30 2011
   - T(n,k) is the number of lattice paths of length n, starting from the origin and ending at (n,k), using horizontal steps H=(1,0), up steps U=(1,1) and down steps D=(1,-1), never containing UUU, DD, HD. For instance, for n=4 and k=2, we have the paths; HHUU, HUHU, HUUH, UHHU, UHUH, UUHH, UUDU, UDUU, UUUD. - _Emanuele Munarini_, Mar 15 2011
   - Row sums form Pell numbers <a href="http://oeis.org/A000129">A000129</a>, T(n,0) forms Fibonacci numbers <a href="http://oeis.org/A000045">A000045</a>, T(n,1) forms <a href="http://oeis.org/A001629">A001629</a>. T(n+k,n-k) is polynomial sequence of degree k.
   - T(n,k) gives a convolved Fibonacci sequence (<a href="http://oeis.org/A001629">A001629</a>, <a href="http://oeis.org/A001872">A001872</a>, etc.).
   - As a Riordan array, this is (1/(1-x-x^2),x/(1-x-x^2)). An interesting factorization is (1/(1-x^2),x/(1-x^2))*(1/(1-x),x/(1-x)) [abs(<a href="http://oeis.org/A049310">A049310</a>) times <a href="http://oeis.org/A007318">A007318</a>]. Diagonal sums are the Jacobsthal numbers <a href="http://oeis.org/A001045">A001045</a>(n+1). - _Paul Barry_, Jul 28 2005
   - T(n,k) = T'(n+1,k+1), T' given by [0, 1, 1, -1, 0, 0, 0, 0, 0, 0, 0, ...] DELTA [1, 0, 0, 0, 0, 0, 0, 0, 0, ...] where DELTA is the operator defined in <a href="http://oeis.org/A084938">A084938</a>. - _Philippe Deléham_, Nov 19 2005
   - Equals <a href="http://oeis.org/A049310">A049310</a> * <a href="http://oeis.org/A007318">A007318</a> as infinite lower triangular matrices. - _Gary W. Adamson_, Oct 28 2007
   - This triangle may also be obtained from the coefficients of the Morgan-Voyce polynomials defined by: Mv(x, n) = (x + 1)*Mv(x, n - 1) + Mv(x, n - 2). - _Roger L. Bagula_, Apr 09 2008
   - Row sums are <a href="http://oeis.org/A000129">A000129</a>. - _Roger L. Bagula_, Apr 09 2008
   - Absolute value of coefficients of the characteristic polynomial of tridiagonal matrices with 1's along the main diagonal, and i's along the superdiagonal and the subdiagonal (where i=sqrt(-1), see Mathematica program). - _John M. Campbell_, Aug 23 2011

_Formulae_:
   - T(n, m) = T'(n-1, m)+T'(n-2, m)+T'(n-1, m-1), where T'(n, m) = T(n, m) for n >= 0 and 0< = m< = n and T'(n, m) = 0 otherwise.
   - G.f.: 1/(1 - y - y*z - y^2).
   - G.f. for k-th column: x/(1-x-x^2)^k.
   - T(n, m) = sum(binomial(m+k, m)*binomial(k, n-k-m), k=0..n-m), n>=m>=0, else 0. - _Wolfdieter Lang_, Jun 17 2002
   - T(n, m) = ((n-m+1)*T(n, m-1) + 2*(n+m)*T(n-1, m-1))/(5*m), n >= m >= 1; T(n, 0)= <a href="http://oeis.org/A000045">A000045</a>(n+1); T(n, m)= 0 if n<m. - _Wolfdieter Lang_, Apr 12 2000
   - Chebyshev coefficient triangle (abs(<a href="http://oeis.org/A049310">A049310</a>)) times Pascal's triangle (<a href="http://oeis.org/A007318">A007318</a>) as product of lower triangular matrices. T(n, k)=sum{k=0..n, C((n+j)/2, j)(1+(-1)^(n+j))C(j, k)/2}. - _Paul Barry_, Dec 22 2004
   - Let R(n) = n-th row polynomial in x, with R(0)=1, then R(n+1)/R(n) equals the continued fraction [1+x;1+x, ...(1+x) occurring (n+1) times..., 1+x] for n>=0. - _Paul D. Hanna_, Feb 27 2004
   - T(n,k) = sum{j=0..n, C(n-j,j)*C(n-2*j,k)}; in Egorychev notation, T(n,k)=res_w(1-w-w^2)^(-k-1)*w^(-n+k+1). - _Paul Barry_, Sep 13 2006
   - Sum_{k=0..n} T(n,k)*x^k = <a href="http://oeis.org/A000045">A000045</a>(n+1), <a href="http://oeis.org/A000129">A000129</a>(n+1), <a href="http://oeis.org/A006190">A006190</a>(n+1), <a href="http://oeis.org/A001076">A001076</a>(n+1), <a href="http://oeis.org/A052918">A052918</a>(n), <a href="http://oeis.org/A005668">A005668</a>(n+1), <a href="http://oeis.org/A054413">A054413</a>(n), <a href="http://oeis.org/A041025">A041025</a>(n), <a href="http://oeis.org/A099371">A099371</a>(n+1), <a href="http://oeis.org/A041041">A041041</a>(n), <a href="http://oeis.org/A049666">A049666</a>(n+1), <a href="http://oeis.org/A041061">A041061</a>(n), <a href="http://oeis.org/A140455">A140455</a>(n+1), <a href="http://oeis.org/A041085">A041085</a>(n), <a href="http://oeis.org/A154597">A154597</a>(n+1), <a href="http://oeis.org/A041113">A041113</a>(n) for n = 0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15 respectively. - _Philippe Deléham_, Nov 29 2009
   - T((m+1)*n+r-1,m*n+r-1)*r/(m*n+r) = Sum_{k=1..n} k/n*T((m+1)*n-k-1,m*n-1)*(r+k,r), n>=m>1.
   - T(n-1,m-1) = m/n*sum(k=1..n-m+1,k*<a href="http://oeis.org/A000045">A000045</a>(k)*T(n-k-1,m-2),k,1,n-m+1), n>=m>1. - _Vladimir Kruchinin_, Mar 17 2011
   - T(n,k) = binomial(n,k)*hypergeom([(k-n)/2, (k-n+1)/2], [-n], -4) for n>=1. - _Peter Luschny_, Apr 25 2016

_Cross references_:
   - <a href="http://oeis.org/A038112">A038112</a>(n)=T(2n, n). <a href="http://oeis.org/A038137">A038137</a> is reflected version. Maximal row entries: <a href="http://oeis.org/A038149">A038149</a>.
   - Diagonal differences are in <a href="http://oeis.org/A055830">A055830</a>. Vertical sums are in <a href="http://oeis.org/A091186">A091186</a>.
   - Cf. <a href="http://oeis.org/A007318">A007318</a>, <a href="http://oeis.org/A049310">A049310</a>, <a href="http://oeis.org/A000129">A000129</a>, <a href="http://oeis.org/A155161">A155161</a>, <a href="http://oeis.org/A122542">A122542</a>, <a href="http://oeis.org/A059283">A059283</a>, <a href="http://oeis.org/A228196">A228196</a>, <a href="http://oeis.org/A228576">A228576</a>.
   - Some other Fibonacci-Pascal triangles: <a href="http://oeis.org/A027926">A027926</a>, <a href="http://oeis.org/A036355">A036355</a>, <a href="http://oeis.org/A074829">A074829</a>, <a href="http://oeis.org/A105809">A105809</a>, <a href="http://oeis.org/A109906">A109906</a>, <a href="http://oeis.org/A111006">A111006</a>, <a href="http://oeis.org/A114197">A114197</a>, <a href="http://oeis.org/A162741">A162741</a>, <a href="http://oeis.org/A228074">A228074</a>.


<hr><div align='center'><b><a href='http://oeis.org/A130595'>A130595</a></b>: Triangle read by rows: lower triangular matrix which is inverse to Pascal's triangle (A007318) regarded as a lower triangular matrix.<br></div>

by _Philippe Deléham_, Jun 17 2007

_Keywords_: `sign,nice,tabl`

_Data_:

$$
\begin{array}{c|ccccccccccc }
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & -1 & 1 &  &  &  &  &  &  &  & \\2 & 1 & -2 & 1 &  &  &  &  &  &  & \\3 & -1 & 3 & -3 & 1 &  &  &  &  &  & \\4 & 1 & -4 & 6 & -4 & 1 &  &  &  &  & \\5 & -1 & 5 & -10 & 10 & -5 & 1 &  &  &  & \\6 & 1 & -6 & 15 & -20 & 15 & -6 & 1 &  &  & \\7 & -1 & 7 & -21 & 35 & -35 & 21 & -7 & 1 &  & \\8 & 1 & -8 & 28 & -56 & 70 & -56 & 28 & -8 & 1 & \\9 & -1 & 9 & -36 & 84 & -126 & 126 & -84 & 36 & -9 & 1
\end{array}
$$
        

_Comments_:
   - Triangle T(n,k), read by rows, given by [-1,0,0,0,0,0,0,0,...] DELTA [1,0,0,0,0,0,0,0,...] where DELTA is the operator defined in <a href="http://oeis.org/A084938">A084938</a>.
   - Coefficients of the polynomials generated by the e.g.f. exp(x*t)*exp(-t). - _Peter Luschny_, Jul 13 2009
   - Riordan array (1/(1+x), x/(1+x)). - _Philippe Deléham_, Nov 29 2009
   - Multiplication of a sequence (written as column vector) by this matrix (to the left) yields the inverse Binomial transform of the sequence. - _M. F. Hasler_, Nov 01 2014
   - From _Tom Copeland_, Nov 16 2016: 
       - This signed Pascal matrix IP and the Pascal matrix P contain the coefficients of a prototypical pair of Appell polynomial sequences that are inverse under umbral composition with e.g.f.s e^((x-1)*t) = e^(-t) e^(xt) = f(t) e^(xt) and e^((x+1)t) = e^t e^(xt) = g(t) e^(xt) and row polynomials q_n(x) = (x-1)^n and p_n(x) = (x+1)^n, respectively. The inverse property for an Appell pair is reflected in IP*P = identity matrix, f(t) = 1/g(t), the umbral relation p_n(q.(x)) = x^n = q_n(p.(x)), and their respective raising operators R_(Ip) = x - h(D) and R_P = x + h(D) differing only in the sign of the differential term (h(D) = 1, in this case). The lowering operator for an Appell sequence is L = D = d/dx with L p_n(x) = n*p_(n-1)(x), and the raising operator is defined by R p_n(x) = p_(n+1)(x).
       - The related signed Pascal matrix M with M(n,k) = (-1)^n IP(n,k)  = (-1)^k P(n,k) has the e.g.f. e^((1-x)t) = e^t e^(-xt), and w_n(x) = (1-x)^n is not an Appell sequence, but it is a Sheffer sequence with lowering and raising operators L = -D and R = 1 - x, and M = M^(-1) since w_n(w.(x)) = (1-w.(x))^n = sum_{k = 0,..,n} binomial(n,k) (-1)^k w_k(x) = (1-(1-x))^n = x^n.
       - Umbral composition of a pair of Sheffer polynomial sequences, of which Appell sequences are a special class, is equivalent to the multiplication of their respective coefficient matrices.

_Formulae_:
   - T(n,k) = (-1)^(n-k)*binomial(n,k) = (-1)^(n-k)*<a href="http://oeis.org/A007318">A007318</a>(n,k).
   - T(n,k) = T(n-1,k-1)-T(n-1,k). - _Philippe Deléham_, Oct 10 2011
   - G.f.: 1/(1+x-x*y). - _R. J. Mathar_, Aug 11 2015 [corrected by _Anders Claesson_, Nov 28 2015]
   - Conjecture from _Dale Gerdemann_, Nov 28 2015: T(n,k) = (n-k+1)*T(n-1,k-1) + (k-1)*T(n-1,k). Proof from _Anders Claesson_, Nov 29 2015: It follows from T(n,k) = T(n-1,k-1) - T(n-1,k) and n*T(n-1,k-1) = k*T(n,k) that (n-k+1)*T(n-1,k-1) + (k-1)*T(n-1,k) = n*T(n-1,k-1) - (k-1)*T(n-1,k-1) + (k-1)*T(n-1,k) = n*T(n-1,k-1) - (k-1)*(T(n-1,k-1) - T(n-1,k)) = n*T(n-1,k-1) - (k-1)*T(n,k) = n*T(n-1,k-1) - k*T(n,k) + T(n,k) = T(n,k). QED
   - (-1)^(n+1) Sum_{k=1..n} T(n,k)/k = Sum_{k=1..n} 1/k = H(n) where H(n) is the n-th harmonic number. For a proof see link "Relation between binomial coefficients and harmonic numbers". - _Wolfgang Hintze_, Oct 22 2016

_Cross references_:
   - Cf. <a href="http://oeis.org/A007318">A007318</a>.


<hr><div align='center'><b><a href='http://oeis.org/A132440'>A132440</a></b>: Infinitesimal Pascal matrix: generator (lower triangular matrix representation) of the Pascal matrix, the classical operator xDx, iterated Laguerre transforms, associated matrices of the list partition transform and general Euler transformation for sequences.<br></div>

by _Tom Copeland_, Nov 13 2007, Nov 15 2007, Nov 22 2007, Dec 02 2007

_Keywords_: `easy,nonn,tabl`

_Data_:

$$
\begin{array}{c|ccccccccccc }
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 0 &  &  &  &  &  &  &  &  & \\1 & 1 & 0 &  &  &  &  &  &  &  & \\2 & 0 & 2 & 0 &  &  &  &  &  &  & \\3 & 0 & 0 & 3 & 0 &  &  &  &  &  & \\4 & 0 & 0 & 0 & 4 & 0 &  &  &  &  & \\5 & 0 & 0 & 0 & 0 & 5 & 0 &  &  &  & \\6 & 0 & 0 & 0 & 0 & 0 & 6 & 0 &  &  & \\7 & 0 & 0 & 0 & 0 & 0 & 0 & 7 & 0 &  & \\8 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 8 & 0 & \\9 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 9 & 0
\end{array}
$$
        

_Comments_:
   - Let M(t) = exp(t*T) = Lim_{n->infinity} (1 + t*T/n)^n.
   - Pascal matrix = [ binomial(n,k) ] = M(1) = exp(T), truncating the series gives the n X n submatrices.
   - Inverse Pascal matrix = M(-1) = exp(-T) = matrix for inverse binomial transform.
   - A(j) = T^j / j! equals the matrix [bin(n,k) * delta(n-k-j)] where delta(n) = 1 if n=0 and vanishes otherwise (Kronecker delta); i.e., A(j) is a matrix with all the terms 0 except for the j-th lower (or main for j=0) diagonal which equals that of the Pascal triangle. Hence the A(j)'s form a linearly independent basis for all matrices of the form [binomial(n,k) d(n-k)] which include as a subset the invertible associated matrices of the list partition transform (LPT) of <a href="http://oeis.org/A133314">A133314</a>.
   - For sequences with b(0) = 1, umbrally,
   - M[b(.)] = exp(b(.)*T) = [ binomial(n,k) * b(n-k) ] = matrices associated to b by LPT.
   - [M[b(.)]]^(-1) = exp(c(.)*T) = [ binomial(n,k) * c(n-k) ] = matrices associated to c, where c = LPT(b) . Or,
   - [M[b(.)]]^(-1) = exp[LPT(b(.))*T] = LPT[M(b(.))] = M[LPT(b(.))]= M[c(.)].
   - This is related to xDx, the iterated Laguerre transform and the general Euler transformation of a sequence through the comments in <a href="http://oeis.org/A132013">A132013</a> and <a href="http://oeis.org/A132014">A132014</a> and the relation [Sum_{k=0..n} binomial(n,k) * b(n-k) * d(k)] = M(b)*d, (n-th term). See also <a href="http://oeis.org/A132382">A132382</a>.
   - If b(n,x) is a binomial type Sheffer sequence, then M[b(.,x)]*s(y) = s(x+y) when s(y) = (s(0,y),s(1,y),s(2,y),...) is an array for a Sheffer sequence with the same delta operator as b(n,x) and [M[b(.,x)]]^(-1) is given by the formulas above with b(n) replaced by b(n,x) as b(0,x)=1 for a binomial type Sheffer sequence.

_Formulae_:
   - T = log(P) with the Pascal matrix P:=<a href="http://oeis.org/A007318">A007318</a>. This should be read as T_N = log(P_N) with P_N the N X N matrix P, N>=2. Because P_N is lower triangular with all diagonal elements 1, the series log(1_N-(1_N-P_N)) stops after N-1 terms because (1_N-P_N)^N is the 0_N-matrix. - _Wolfdieter Lang_, Oct 14 2010
   - Given a polynomial sequence p_n(x) with p_0(x)=1 and the lowering and raising operators L and R defined by L p_n(x) = n * p_(n-1)(x) and R p_n(x) = p_(n+1)(x), the matrix T represents the action of R*L*R in the p_n(x) basis. For p_n(x) = x^n, L = D = d/dx and R = x. For p_n(x) = x^n/n!, L = DxD and R = D^(-1). - _Tom Copeland_, Oct 25 2012
   - From _Tom Copeland_, Apr 26 2014: 
       - A) T = exp(<a href="http://oeis.org/A238385">A238385</a>-I) - I
       - B)   = [St1]*P*[St2] - I
       - C)   = [St1]*P*[St1]^(-1) - I
       - D)   = [St2]^(-1)*P*[St2] - I
       - E)   = [St2]^(-1)*P*[St1]^(-1) - I
       - where P=<a href="http://oeis.org/A007318">A007318</a>, [St1]=padded <a href="http://oeis.org/A008275">A008275</a> just as [St2]=<a href="http://oeis.org/A048993">A048993</a>=padded <a href="http://oeis.org/A008277">A008277</a>, and I=identity matrix. 
   - From _Robert Israel_, Oct 02 2015: 
       - G.f. Sum_{k >= 1} k x^((k+3/2)^2/2 - 17/8) is related to Jacobi theta functions.
       - If 8*n+17 = y^2 is a square, then a(n) = (y-3)/2, otherwise a(n) = 0. 



<hr><div align='center'><b><a href='http://oeis.org/A083093'>A083093</a></b>: Triangle formed by reading Pascal's triangle (A007318) mod 3.<br></div>

by _Benoit Cloitre_, Apr 22 2003

_Keywords_: `easy,nonn,tabl`

_Data_:

$$
\begin{array}{c|ccccccccccc }
n, k & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 \\
\hline
0 & 1 &  &  &  &  &  &  &  &  & \\1 & 1 & 1 &  &  &  &  &  &  &  & \\2 & 1 & 2 & 1 &  &  &  &  &  &  & \\3 & 1 & 0 & 0 & 1 &  &  &  &  &  & \\4 & 1 & 1 & 0 & 1 & 1 &  &  &  &  & \\5 & 1 & 2 & 1 & 1 & 2 & 1 &  &  &  & \\6 & 1 & 0 & 0 & 2 & 0 & 0 & 1 &  &  & \\7 & 1 & 1 & 0 & 2 & 2 & 0 & 1 & 1 &  & \\8 & 1 & 2 & 1 & 2 & 1 & 2 & 1 & 2 & 1 & \\9 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1
\end{array}
$$
        

_Comments_:
   - Start with [1], repeatedly apply the map 0 -> [000/000/000], 1 -> [111/120/100], 2 -> [222/210/200] . [_Philippe Deléham_, Apr 16 2009]

_Formulae_:
   - T(i, j) = binomial(i, j) mod 3.
   - T(n+1,k) = (T(n,k) + T(n,k-1)) mod 3. - _Reinhard Zumkeller_, Jul 11 2013

_Cross references_:
   - Cf. <a href="http://oeis.org/A007318">A007318</a>, <a href="http://oeis.org/A051638">A051638</a> (row sums), <a href="http://oeis.org/A090044">A090044</a>, <a href="http://oeis.org/A047999">A047999</a>, <a href="http://oeis.org/A034931">A034931</a>, <a href="http://oeis.org/A034930">A034930</a>, <a href="http://oeis.org/A008975">A008975</a>, <a href="http://oeis.org/A034932">A034932</a>, <a href="http://oeis.org/A062296">A062296</a>, <a href="http://oeis.org/A006047">A006047</a>.
   - Cf. <a href="http://oeis.org/A006996">A006996</a> (central terms), <a href="http://oeis.org/A173019">A173019</a>, <a href="http://oeis.org/A206424">A206424</a>, <a href="http://oeis.org/A227428">A227428</a>.



## Sandbox

In [60]:
from requests import get
from IPython.display import Markdown

In [61]:
code=45

payload = {"fmt": "json", 
           #"q": "id:A{:06d}".format(code)}
           "q": "fibonacci",
           "start": 0}

doc_result = get("https://oeis.org/search", params=payload,)
doc = doc_result.json()

In [62]:
doc_result.url

'https://oeis.org/search?fmt=json&q=fibonacci&start=0'

In [63]:
doc.keys()

dict_keys(['greeting', 'query', 'count', 'start', 'results'])

In [64]:
results = doc['results']
results[0]['comment'][:10]

["Also sometimes called Lamé's sequence.",
 "F(n+2) = number of binary sequences of length n that have no consecutive 0's.",
 'F(n+2) = number of subsets of {1,2,...,n} that contain no consecutive integers.',
 'F(n+1) = number of tilings of a 2 X n rectangle by 2 X 1 dominoes.',
 'F(n+1) = number of matchings (i.e., Hosoya index) in a path graph on n vertices: F(5)=5 because the matchings of the path graph on the vertices A, B, C, D are the empty set, {AB}, {BC}, {CD} and {AB, CD}. - _Emeric Deutsch_, Jun 18 2001',
 'F(n) = number of compositions of n+1 with no part equal to 1. [Cayley, Grimaldi]',
 'Positive terms are the solutions to z = 2*x*y^4 + (x^2)*y^3 - 2*(x^3)*y^2 - y^5 - (x^4)*y + 2*y for x,y >= 0 (Ribenboim, page 193). When x=F(n), y=F(n + 1) and z>0 then z=F(n + 1).',
 'For Fibonacci search see Knuth, Vol. 3; Horowitz and Sahni; etc.',
 "F(n) is the diagonal sum of the entries in Pascal's triangle at 45 degrees slope. - _Amarnath Murthy_, Dec 29 2001",
 'F(n+1) is the numbe

In [65]:
Markdown(pretty_print(doc['results'][0], 
                      comment=lambda i,c: "binomial" in c,
                      formula=lambda i,f: "theorem" in f,
                      link=lambda i,l: "paul barry" in l,
                      reference=lambda i,r: "riordan" in r))

<div align='center'><b><a href='http://oeis.org/A000045'>A000045</a></b>: Fibonacci numbers: F(n) = F(n-1) + F(n-2) with F(0) = 0 and F(1) = 1.<br></div>

by _N. J. A. Sloane_, 1964

_Keywords_: `nonn,core,nice,easy,hear`

_Data_:

$$
\begin{array}{c|ccccccccccccccc }
    n & 0 & 1 & 2 & 3 & 4 & 5 & 6 & 7 & 8 & 9 & 10 & 11 & 12 & 13 & 14 \\
    \hline
    A000045(n) & 0 & 1 & 1 & 2 & 3 & 5 & 8 & 13 & 21 & 34 & 55 & 89 & 144 & 233 & 377
\end{array}
$$
        

_Comments_:
   - Fib(n+2) = Sum_{k=0..n} binomial(floor((n+k)/2),k), row sums of <a href="http://oeis.org/A046854">A046854</a>. - _Paul Barry_, Mar 11 2003
   - The sequence F(n) is the binomial transformation of the alternating sequence (-1)^(n-1)*F(n), whereas the sequence F(n+1) is the binomial transformation of the alternating sequence (-1)^n*F(n-1). Both of these facts follow easily from the equalities a(n;1)=F(n+1) and b(n;1)=F(n) where a(n;d) and b(n;d) are so-called "delta-Fibonacci" numbers as defined in comments to <a href="http://oeis.org/A014445">A014445</a> (see also the papers of Witula et al.). - _Roman Witula_, Jul 24 2012
   - From _Tom Copeland_, Nov 02 2014: 
       - Let P(x) = x/(1+x) with comp. inverse Pinv(x) = x/(1-x) = -P[-x], and C(x)= [1-sqrt(1-4x)]/2, an o.g.f. for the shifted Catalan numbers <a href="http://oeis.org/A000108">A000108</a>, with inverse Cinv(x) = x * (1-x).
       - Fin(x) = P[C(x)] = C(x)/[1 + C(x)] is an o.g.f. for the Fine numbers, <a href="http://oeis.org/A000957">A000957</a> with inverse Fin^(-1)(x) = Cinv[Pinv(x)] = Cinv[-P(-x)].
       - Mot(x) = C[P(x)] = C[-Pinv(-x)] gives an o.g.f. for shifted <a href="http://oeis.org/A005043">A005043</a>, the Motzkin or Riordan numbers with comp. inverse Mot^(-1)(x) = Pinv[Cinv(x)] = (x - x^2) / (1 - x + x^2) (cf. <a href="http://oeis.org/A057078">A057078</a>).
       - BTC(x) = C[Pinv(x)] gives <a href="http://oeis.org/A007317">A007317</a>, a binomial transform of the Catalan numbers, with BTC^(-1)(x) = P[Cinv(x)].
       - Fib(x) = -Fin[Cinv(Cinv(-x))] = -P[Cinv(-x)] = x + 2 x^2 + 3 x^3 + 5 x^4 + ... = (x+x^2)/[1-x-x^2] is an o.g.f. for the shifted Fibonacci sequence <a href="http://oeis.org/A000045">A000045</a>, so the comp. inverse is Fib^(-1)(x) = -C[Pinv(-x)] = -BTC(-x) and Fib(x) = -BTC^(-1)(-x).
       - Generalizing to P(x,t) = x /(1 + t*x) and Pinv(x,t) = x /(1 - t*x) = -P(-x,t) gives other relations to lattice paths, such as the o.g.f. for <a href="http://oeis.org/A091867">A091867</a>, C[P[x,1-t]], and that for <a href="http://oeis.org/A104597">A104597</a>, Pinv[Cinv(x),t+1].

_Formulae_:
   - A sufficient condition for F(m) to be divisible by a prime p is (p - 1) divides m, if p == 1 or 4 (mod 5); (p + 1) divides m, if p == 2 or 3 (mod 5); or 5 divides m, if p = 5. (This is essentially Theorem 180 in Hardy and Wright.) - Fred W. Helenius (fredh(AT)ix.netcom.com), Jun 29 2001
   - _Kurmang. Aziz. Rashid_, Feb 21 2004, makes 4 conjectures and gives 3 theorems:
   - Theorem 1: for n>=0, {F(n+3)^ 2 - F(n+1)^ 2}/F(n+2)={F(n+3)+ F(n+1)}.
   - Theorem 2: for n>=0, F(n+10) = 11*F(n+5) + F(n).
   - Theorem 3: for n>=6, F(n) = 4*F(n-3) + F(n-6).

_Cross references_:
   - Cf. <a href="http://oeis.org/A039834">A039834</a> (signed Fibonacci numbers), <a href="http://oeis.org/A001690">A001690</a> (complement), <a href="http://oeis.org/A000213">A000213</a>, <a href="http://oeis.org/A000288">A000288</a>, <a href="http://oeis.org/A000322">A000322</a>, <a href="http://oeis.org/A000383">A000383</a>, <a href="http://oeis.org/A060455">A060455</a>, <a href="http://oeis.org/A030186">A030186</a>, <a href="http://oeis.org/A020695">A020695</a>, <a href="http://oeis.org/A020701">A020701</a>, <a href="http://oeis.org/A071679">A071679</a>, <a href="http://oeis.org/A099731">A099731</a>, <a href="http://oeis.org/A100492">A100492</a>, <a href="http://oeis.org/A094216">A094216</a>, <a href="http://oeis.org/A094638">A094638</a>, <a href="http://oeis.org/A000108">A000108</a>, <a href="http://oeis.org/A101399">A101399</a>, <a href="http://oeis.org/A101400">A101400</a>, <a href="http://oeis.org/A001611">A001611</a>, <a href="http://oeis.org/A000071">A000071</a>, <a href="http://oeis.org/A157725">A157725</a>, <a href="http://oeis.org/A001911">A001911</a>, <a href="http://oeis.org/A157726">A157726</a>, <a href="http://oeis.org/A006327">A006327</a>, <a href="http://oeis.org/A157727">A157727</a>, <a href="http://oeis.org/A157728">A157728</a>, <a href="http://oeis.org/A157729">A157729</a>, <a href="http://oeis.org/A167616">A167616</a>, <a href="http://oeis.org/A059929">A059929</a>, <a href="http://oeis.org/A144152">A144152</a>, <a href="http://oeis.org/A152063">A152063</a>, <a href="http://oeis.org/A114690">A114690</a>, <a href="http://oeis.org/A003893">A003893</a>, <a href="http://oeis.org/A000032">A000032</a>, <a href="http://oeis.org/A060441">A060441</a>, <a href="http://oeis.org/A000930">A000930</a>, <a href="http://oeis.org/A003269">A003269</a>, <a href="http://oeis.org/A000957">A000957</a>, <a href="http://oeis.org/A057078">A057078</a>, <a href="http://oeis.org/A007317">A007317</a>, <a href="http://oeis.org/A091867">A091867</a>, <a href="http://oeis.org/A104597">A104597</a>, <a href="http://oeis.org/A249548">A249548</a>, <a href="http://oeis.org/A262342">A262342</a>, <a href="http://oeis.org/A001060">A001060</a>, <a href="http://oeis.org/A022095">A022095</a>.
   - First row of arrays <a href="http://oeis.org/A103323">A103323</a>, <a href="http://oeis.org/A234357">A234357</a>. Second row of arrays <a href="http://oeis.org/A099390">A099390</a>, <a href="http://oeis.org/A048887">A048887</a>, and <a href="http://oeis.org/A092921">A092921</a> (k-generalized Fibonacci numbers).
   - a(n) = <a href="http://oeis.org/A094718">A094718</a>(4, n). a(n) = <a href="http://oeis.org/A101220">A101220</a>(0, j, n).
   - a(n) = <a href="http://oeis.org/A090888">A090888</a>(0, n+1) = <a href="http://oeis.org/A118654">A118654</a>(0, n+1) = <a href="http://oeis.org/A118654">A118654</a>(1, n-1) = <a href="http://oeis.org/A109754">A109754</a>(0, n) = <a href="http://oeis.org/A109754">A109754</a>(1, n-1), for n > 0.
   - Fibonacci-Pascal triangles: <a href="http://oeis.org/A027926">A027926</a>, <a href="http://oeis.org/A036355">A036355</a>, <a href="http://oeis.org/A037027">A037027</a>, <a href="http://oeis.org/A074829">A074829</a>, <a href="http://oeis.org/A105809">A105809</a>, <a href="http://oeis.org/A109906">A109906</a>, <a href="http://oeis.org/A111006">A111006</a>, <a href="http://oeis.org/A114197">A114197</a>, <a href="http://oeis.org/A162741">A162741</a>, <a href="http://oeis.org/A228074">A228074</a>.
   - Boustrophedon transforms: <a href="http://oeis.org/A000738">A000738</a>, <a href="http://oeis.org/A000744">A000744</a>.
   - Powers: <a href="http://oeis.org/A103323">A103323</a>, <a href="http://oeis.org/A105317">A105317</a>, <a href="http://oeis.org/A254719">A254719</a>.
   - Numbers of prime factors: <a href="http://oeis.org/A022307">A022307</a> and <a href="http://oeis.org/A038575">A038575</a>.
   - Cf. <a href="http://oeis.org/A163733">A163733</a>.

_Links_:
   - Paul Barry, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL8/Barry/barry84.html">A Catalan Transform and Related Transformations on Integer Sequences</a>, Journal of Integer Sequences, Vol. 8 (2005), Article 05.4.5.
   - Paul Barry, <a href="http://www.cs.uwaterloo.ca/journals/JIS/VOL9/Barry/barry91.html">On Integer-Sequence-Based Constructions of Generalized Pascal Triangles</a>, Journal of Integer Sequences, Vol. 9 (2006), Article 06.2.4.

_References_:
   - J. Riordan, An Introduction to Combinatorial Analysis, Princeton University Press, Princeton, NJ, 1978.

---
<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.